# Cosmological Structure Growth

In [ ]:
# @title
""" VERSION HISTORY
LCDM-v2
2025-12-16 Converted to Colab compatible Jupyter notebook
DFS-v11-1
2025-11-02 Transformation to individual masses for particles. Use of grad_phi5 module
DFS-v11-0
2025-10-31 Integrated deposit and gather functions from grad_phi-v4 module
2025-10-28 Integrated grad_phi-v3.cpp, leading to performance improvements in combined_step, cooling_heating_step and
           refine_potential_cpp functions
2025-10-23 Introduced refine_subgrid function from grad_phi module, parallelizing the DST calculation, leadimng to
           performance improvements
2025-10-22 Introduced time subcycling in fine grid. Consolidated everything in combined_step_subcycling function
2025-10-18 Introduced thermo mesh refinement
2025-10-14 Introduced self shielding in cooling_heating_step and restructured that function fundamentally
           Moved calculation of max_tot_od from cooling_heating_step to pm_acceleration (that is where it naturally belongs)
2025-10-11 Re-integrated Python version of cooling_heating_step to prepare for integration of thermo mesh refinement
2025-10-09 Added gravity mesh refinement to leapfrog_step_py
           Re-integrated Python version of leapfrog_step to prepare for integration of gravity mesh refinement
2025-10-02 Introduced Tinker benchmark for number density per halo weight class, replacing Press-Schechter
2025-09-30 Fully integrated use_TSC
2025-09-29 Integrated combined_step function, leading to massive performance improvement
2025-09-25 Started use of CODEX
           Turned cooling_heating_step from Python to C++ using CODDEX. Some performance improvement, but not stellar
DFS-v10-comoving
2025-09-24 Extended Artificial Viscosity function to better preserve angular momentum in disk midplanes
           Fixed critical error in Artificial Viscosity function (use of dx_phys where necessary)
2025-09-21 Fixed Layzer-Irvine energy equation
2025-09-17 Introduced Wiersma collision tables for z > 8.989. This leads to realistic temperatures for pre-reionization epoch
2025-09-11 Introduced exponential update of u_grid
2025-08-30 Integrated Wiersma cooling tables. More realistic temperature evolment.
2025-08-28 Accelerated thermo calculations
2025-08-25 Animation and illustration of Thermodynamic processes
2025-08-24 First realistic results of Themodynamic processes (heating, colling, pressure forces).
2025-08-22 Early steps integrating Thermodynamics
DFS-v9-comoving
2025-08-19 Corrected error in initialization of M_total
2025-08-15 Weak-lensing convergence kappa analysis added
2025-08-12 First run with 256**3 = 16'777k particles and cells. Time ca 42min.
2025-08-12 Transformed pm_acceleration function to cpp, including parallel FFT processing. Factor 2 performance gain on 2097 kBodies.
           Almost identical results
2025-08-11 Implemented TSC approach to density calculation. Minimal changes to results, but lower performance (timewise)
           Further improved variable consistency
2025-08-07 Added html summary report
2025-08-06 Adopted simple cooling model for baryons, only dependent on the local overdensity. Shows concentration of baryons in halo centers
2025-08-05 Integrated calculation of all LCDM key quantities
2025-08-04 Changed resampling approach for animation to one rate qq applying to each of the 3 space dimensions
2025-08-02 Consolidated code. Focused Halo Analysis on full set of bodies. First simulation of 128**3 bodies and cells in ca. 4 mins.
           Fixed spin calculation in halo analysis. Now realistic spins are produced (around 0.05)
           Added halo statistic (halo count by weight classes, compared to theoretical expectations based on Press-Schechter) and halo visualization
2025-08-01 Added Energy Analysis based on phi
2025-07-31 Optimized PM by migrating delta field calculation (CIC) and mesh interpolation to C++. 64**3 bodies run in 21s.
           Animation changed to show a fraction of the bodies simulated. Calculation of energies disabled as too slow (PE based on body-body interactions)
2025-07-30 Implemented accelerations based on Particle-Mash Method. Results roughly consistent with earlier body-to-body approaches
           A factor of 2**2 when calculating acc and a factor of 2 when calculating potential energy remains unexplained
           PM-Method now considers periodic boundary conditions
2025-07-28 Copy created
DFS-v8-10Mpc-comoving
2025-07-27 Corrected scaling of s2 contribution to displacement and velocity (previously too low by factor 1/D_init)
2025-07-26 Improved charts and code readability
2025-07-24 Extended grad_phi module to calculate energies.
2025-07-23 Introduced C++ based force calculation based on parallel processing. import grad_phi provides the respective functions
           Massive performance improvement
           Enabled the use of compression factors q that do not divide N. Broader ranges of compression possible
           Introduced Barnes Hat force calculation in Python. Too slow to provide practical advantage over brute force
2025-07-22 Background acceleration now determined based on critical mass.
2025-07-22 Migration to comoving coordinates. Run with 17000 bodies
DFS-v7-20Mpc-exp5
2025-07-12 Delta field created directly for a=a_init, via power spectrum matching a_init
           Added tidal tensor velocities to spark spin. No effect
           Extended halo analysis to initial conditions
2025-07-12 Made eps dependent on scale factor
2025-07-10 Eliminated averaging bodies after 2LPT
           Eliminated all filtering of delta field used for 2LPT Initial Conditions
           Time steps dt chosen so that delta_a/a is constant and simulation ends at a=1
           q_rot set to zero
           Full 20Mpc box taken as IC
Delta field simulation-v6
2025-07-09 Further speeding up through parallelization using joblib
Delta field simulation-v5
2025-07-08 Speeding up n-body simulation through parallelization of compute_accelerations method
           Added 2LPT (on top of Zeldovich)
Delta field simulation-v4
2025-07-07 Added halo identification
2025-07-06 Added a gray reference sphere to the animation illustrating the hubble flow
           Corrected the cooling mechanism (only peculiar velocities are cooled)
2025-07-05 Creation of delta field now calibrated to length L today (a=1) instead of a=a_init. This allows choice of different a_init without affecting
           the initial delta field. Previous versions were inconsistent (grid scale inconsistent with displacement field scale). Now a_init=0.05 is a reasonable
           choice (earlier versions were at a_init=0.2 which is too late)
           Volume of initial spherical n-body system now determined as a sphere (was determined as a box initially)
Delta field simulation-v3
2025-07-01 Initial n-body system constrained to a sphere (previously a box). Integrated Zeldovich consistemcy check (density contrast field regression)
2025-06-31 Choosing an excerpt of the density field with mean delta > 0
Delta field simulation-v2
2025-06-30 Integrated Zeldovich consistemcy check (density contrast field regression)
2025-06-29 Added separate treatment for baryons (red dots)
Delta field simulation-v1
2025-06-26 Added div s = -delta test
2025-06-25 Integrated density field calculation with n-body simulation
""";

__Parametrization__

In [ ]:
# @title
import os, sys, importlib.util, subprocess
from pathlib import Path

GITHUB_USER = "JoOechslin"
GITHUB_REPO = "Cosmology-Hackathon"
GITHUB_BRANCH = "main"   # or "master"

BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{GITHUB_REPO}/{GITHUB_BRANCH}"

FILES = [
    "grad_phi5.cpython-310-x86_64-linux-gnu.so",
    "cooling_table_collis_v1.npz",
    "cooling_table_v1.npz"
]

def fetch_from_github(filename):
    if not Path(filename).exists():
        url = f"{BASE_URL}/{filename}"
        print(f"⬇️ Downloading {filename}")
        subprocess.run(["wget", "-q", url, "-O", filename], check=True)
    else:
        print(f"✅ {filename} already present")

for f in FILES:
    fetch_from_github(f)

print("📁 Current directory:", os.listdir())

!apt install libfftw3-dev libfftw3-threads3 libomp-dev
!pip install pybind11
!apt-get update
!apt-get install -y libfftw3-dev

# Load compiled extension
so_file = next(
    f for f in os.listdir()
    if f.startswith("grad_phi5") and f.endswith(".so")
)

spec = importlib.util.spec_from_file_location("grad_phi5", so_file)
grad_phi5 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(grad_phi5)

print("✅ grad_phi5 loaded successfully!")

In [ ]:
import numpy as np

# Cosmology
h = 0.7 # unitless
Ω_r = 8.4e-5 # Radiation density (photons and relativistic neutrinos), unitless
Ω_γ = 2.47e-5 / h**2 # photons only (part of Ω_r)
Ω_m = 0.315 # Matter density, unitless
Ω_b = 0.048 # baryons only (part of Ω_m)
Ω_Λ = 1.0 - Ω_r - Ω_m # # Dark energy density, unitless
Ω_k = 1.0 - Ω_m - Ω_Λ - Ω_r   # Omega for curvature (should be zero for LCDM)
H0 = 100 * h  # H0 in km/s/Mpc

# Box and sampling
L = 20   # box size in Mpc
N = 256  # grid size for creation of the initial delta field
qp = 2    # Resampling factor N_bodies (N/qp)**3
qpa = 4   # Resampling for animation: number of bodies in animation = (N/qg/qpa)**3
qg = 2    # Resampling factor N_cells = (N/qg)**3

# Simulation
a_init = 0.01   # initial scale factor
a_final = 1.0 # final scale factor
steps = 600 # number of time steps
steps_fine = 1200
seed = 141 # Drives the random selection of the initial delta field
use_TSC = True
use_Python = False # True=Python functions for Leapfrog and Cooling/Heating (experimental), False means corresponding cpp function are used (speed)
n_threads = 8 # Number of threads for parallel processing in cpp functions

# Mesh refinement
# For 20Mpc, seed=141, Halo 0
cube_fine = 1.2 # Mpc
center_fine = np.array([6.442, 7.726, 5.769]) #Mpc

# For 20Mpc, seed=99, Halo 0
#cube_fine = 1.2 # Mpc
#center_fine = np.array([1.0, 9.5, 18.3]) #Mpc

# For 10Mpc, seed=141, Halo 0
#cube_fine = 0.6 # Mpc
#center_fine = np.array([3.523,3.528,2.833]) #Mpc

# For 30Mpc, seed=141, Halo 0
#cube_fine = 3.2 # Mpc
#center_fine = np.array([9.140,12.260,8.193]) #Mpc

cube_fine = None # No mesh refinement (performance)
refinement = 16 # Factor
f_smooth = 3 # Smoothing of refined rho grid
od_threshold = 100 # Overdensity at which grid refinement kicks in
subcycles = 8 # Number of time subcycles in fine box
n_jeans = 0.5 # Jeans factor (normally 2-4)
margin = 3.5 # Margin in fine grid cells

__Λ-CDM cosmology__

In [ ]:
# @title
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.integrate import solve_ivp
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from scipy.integrate import quad
from numba import njit, prange

# Physical constants
m_sun = 1.989e30 # kg
Mpc_m = 3.085677581e22 # m/Mpc
Mpc_km = Mpc_m/1000 # km/Mpc
Myr_s = 1e6 * 365 * 24 * 3600 # s/Myr
c = 3e8  # m/s
c_cos = c * Myr_s / Mpc_m # speed of light in pc/yr
G = 6.67430e-11 * m_sun / Mpc_m**3 * Myr_s**2 # Mpc²⋅(Mpc/Myr²) / M_sun
H0_cos = H0/Mpc_km*Myr_s # 1/Myr
rho_crit0 = 3*H0_cos**2/8/np.pi/G # M_sun / Mpc³
Gly_Mpc = 306.3915347309086 # in Mpc

def E(a): # dimensionless Hubble parameter
    return np.sqrt(Ω_r / a**4 + Ω_m / a**3 + Ω_k / a**2 + Ω_Λ)

def D(a): # Linear Growth parameter
    integrand = lambda a_: (a_ * E(a_))**-3
    integral, _ = quad(integrand, 0, a, epsabs=1e-6)
    integral_norm, _ = quad(integrand, 0, 1, epsabs=1e-6)
    D_unnorm = (5/2) * Ω_m * E(a) * integral
    D_norm = (5/2) * Ω_m * E(1) * integral_norm
    return D_unnorm / D_norm

def dLogD_dLoga(a):
    da = 1e-4
    return (np.log(D(a + da)) - np.log(D(a - da))) / (np.log(a + da) - np.log(a - da))

def dD_dt(a):
    da = 1e-4
    return (D(a + da) - D(a - da)) / (2*da) * H_init * a_init

def friedmann(t, y): # Friedmann equation (differential form)
    a = y[0]
    return H0_cos * a * np.sqrt(Ω_r / a**4 + Ω_m / a**3 + Ω_k / a**2 + Ω_Λ)

def reach_today(t, y): # Event: stop when a(t) = 1
    a = y[0]
    return a - 1.0
reach_today.terminal = True  # Stop integration
reach_today.direction = 1    # Only trigger when increasing

# Solve Friedman until a = 1 gives age of observable universe
a0 = 1e-8 # Initial condition: tiny scale factor after Big Bang
t_span = [0, 31000]  # Initial conditions: 0 to ~31 Gyr
sol = solve_ivp(friedmann, t_span, [a0], events=reach_today, max_step=31, rtol=1e-12, atol=1e-12)
t = sol.t     # Extract results
a = sol.y[0]
t_universe = sol.t_events[0][0] # Time at which a = 1

# Compute comoving horizon distance (size of observable universe): d_H(t) = c * ∫ (dt / a) and associated speed
integrand = 1 / a
integrand[0] = integrand[1] # the first value distorts the integration
d_H = c_cos * cumulative_trapezoid(integrand, t, initial=0) # Mpc
v_expansion = H0_cos * d_H[-1] # Expansion speed of observable universe today, Mpc/Myr

# Interpolation of a(t) to find time of decoupling
z_dec = 1089.0 # redshift (1089) is determined by detailed modelling of hydrogen recombination
a_dec = 1 / (1 + z_dec)
a_interp = interp1d(a, t)
t_dec = a_interp(a_dec) #Myr

# Determine sound horizon at decoupling
def c_s(a): # Define sound speed c_s(a) in m/s
    R = (3 * Ω_b) / (4 * Ω_γ) * a
    return c / np.sqrt(3 * (1 + R))
cs_over_a = c_s(a) / a # Evaluate c_s(t) / a(t)
i_dec = np.argmax(a >= a_dec)
r_s = cumulative_trapezoid(cs_over_a[:i_dec+1], t[:i_dec+1] * Myr_s, initial=0)
r_s_dec_m = r_s[-1] # Final sound horizon in meters (comoving distance)
r_s_dec = r_s_dec_m / Mpc_m # in Mpc

# Determine angular scale
def H_z(z):  # Proper H(z)
    return H0_cos * E(1 / (1 + z)) # 1/Myr
def integrand(z):  # c / H(z)
    return c_cos / H_z(z) # Mpc
chi, _ = quad(integrand, 0, z_dec) # comoving distance
theta_s_rad = r_s_dec / chi  # in radians
ell_s = np.pi / theta_s_rad
theta_s_deg = np.degrees(theta_s_rad) # Convert angle to degrees if desired

# Determine Event horizon
def integrand(a):
    return c_cos / (a**2 * H0_cos * E(a)) # Mpc
chi_event, _ = quad(integrand, 1, np.inf) # Integrate from now to z → ∞, Mpc

print(f"Computed age of the universe: {t_universe / 1e3:.1f} Gyr") # should be ca. 13.8 Gyr
print(f"Observable universe radius today (particle horizon): {d_H[-1]/c_cos/1e3:.1f} Gly") # should be ca. 46 Gly
print(f"Event horizon today: {chi_event/c_cos/1e3:.1f} Gly") #should be 16.4 Gly
print(f"Hubble sphere today: {1/H0_cos/1e3:.1f} Gly") #
print(f"Expansion speed of observable universe: {v_expansion/c_cos:.1f} c") # should be ca. 900'000 km/s (3 c)
print(f"Time of CMB decoupling: {t_dec * 1e6:.0f} yr") # should be 380'000 yr
print(f"Sound horizon at decoupling (comoving dist):  {r_s_dec / Gly_Mpc:.3f} Gly ({r_s_dec:.1f} Mpc)") # should be 144.6 Mpc
print(f"Angular size of sound horizon: {theta_s_deg:.2f} degrees")
print()

# Cosmological parameters at initial situation
D_init = D(a_init)
f_init = dLogD_dLoga(a_init)
H_init = H0 * E(a_init)  # in km/s/Mpc
dD_dt_init = dD_dt(a_init)

print("Cosmological parameters at initialization:")
print(f"a_init =     {a_init:.3f} (t={a_interp(a_init):.1f} Myr)")
print(f"D_init =     {D_init:.3f}")
print(f"f_init =     {f_init:.3f}")
print(f"dD_dt_init = {dD_dt_init:.3f}")
print(f"H_init =     {H_init:.3f} km/s/Mpc")

# Plot scale factor and acceleration
plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.subplots_adjust(wspace=0.4)  # Increase horizontal space between subplots
ax1 = plt.gca()
ax1.plot(t / 1e3, a, label='a(t)')
ax1.set_xlabel("Time (Gyr)")
ax1.set_ylabel("Scale Factor a(t)")
ax1.set_title("Expansion of the Universe")
ax1.grid(True)
ax2 = ax1.twinx()
ax2.plot(t / 1e3, H0*E(a), color='orange', linestyle='--', label='H(t)')
ax2.set_ylabel("H(t) [km/s/Mpc]")
ax2.set_ylim(top=1000)
ax2.set_ylim(bottom=0)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

# Plot comoving horizon distance
plt.subplot(1, 2, 2)
plt.plot(t/1e3, d_H/c_cos/1e3)
plt.xlabel("Time (Gyr)")
plt.ylabel("Comoving Horizon (Gly)")
plt.title("Observable Universe Radius Over Time")
plt.grid(True)
plt.show()

__Create power spectrum__

In [ ]:
# @title
try:
    import camb
except ImportError:
    !pip install camb
    import camb
from camb import model
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# CAMB setup for ICs at z=32 (a=0.03)
pars = camb.CAMBparams()
pars.set_cosmology(H0=H0, ombh2=0.022, omch2=0.122)
pars.InitPower.set_params(As=2e-9, ns=0.965)
pars.set_matter_power(redshifts=[1/a_init-1], kmax=10.0)  # Key: z=32, not z=0!
pars.NonLinear = model.NonLinear_none  # Linear power spectrum

results = camb.get_results(pars)
kh, z_pk, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=10.0, npoints=1000)
P_lin = interp1d(kh, pk[0], bounds_error=False, fill_value=0.0)

# 2. Direct power spectrum through Cosmological parameters for comparison model
A_s = 2.1e-9 # unitless
n_s = 0.965 # unitless
k_pivot = 0.05  # 1/Mpc
c_km_s = 300000  # speed of light in km/s

def T(k): # Eisenstein & Hu transfer function (no baryons)
    q = k / (Ω_m * h**2)
    L0 = np.log(2*np.e + 1.8*q)
    C0 = 14.2 + 731.0 / (1 + 62.5*q)
    return L0 / (L0 + C0 * q**2) # unitless

def P_3D(k): # 3D power spectrum P(k) from analytical model. k in 1/Mpc
    P_zeta = A_s * (k / k_pivot)**(n_s - 1) # unitless
    T2 = T(k)**2 # unitless
    factor = (c_km_s * k / H0)**2 # unitless
    return 0.25 * (2 * np.pi**2 / k**3) * P_zeta * T2 * factor**2 # Mpc^-3, unexplained factor 0.25 to match CAMB power spectrum

def get_analytic_power_interp(kmin=1e-4, kmax=10.0, num=1000):
    ks = np.logspace(np.log10(kmin), np.log10(kmax), num)
    pk = P_3D(ks) * D_init**2
    return interp1d(ks, pk, bounds_error=False, fill_value=0.0)

P_direct = get_analytic_power_interp()

def ratio_func(k): # ratio b/w direct and CAMB power spectrum
    return P_direct(k)/P_lin(k)

# 3. Plot the power spectrum
plt.figure(figsize=(8, 5))
plt.loglog(kh, P_lin(kh), label="CAMB")
plt.loglog(kh, P_direct(kh), label="Direct", linestyle='--')
plt.loglog(kh, ratio_func(kh), label="Direct/CAMB", linestyle='--')
plt.xlabel("k [1/Mpc]")
plt.ylabel("P(k) [Mpc$^3$]")
plt.title(f"Linear Matter Power Spectrum at z={1/a_init-1:.0f}")
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

__Create delta field__

In [ ]:
# @title
# 4. Build the k-grid and field generator
def make_kgrid(N, L):
    kf = np.fft.fftfreq(N, d=L/N) * 2*np.pi
    kx = kf[:, None, None]
    ky = kf[None, :, None]
    kz = kf[None, None, :]
    return np.sqrt(kx**2 + ky**2 + kz**2), kx, ky, kz # 1/Mpc

def generate_gaussian_field(P_fn, N, L, seed=None):
    if seed is not None:
        np.random.seed(seed)
    kgrid, kx, ky, kz = make_kgrid(N, L) # 1/Mpc
    Pk = P_fn(kgrid) # Mpc^-3
    volume = L**3 # Mpc^3
    dx = L/N
    noise = np.random.normal(size=(N,N,N)) + 1j * np.random.normal(size=(N,N,N))
    delta_k = noise * np.sqrt(Pk * volume / 2.0) #unitless
    dx = L/N # unitless, since N is taken as Mpc in this context
    delta = np.fft.ifftn(delta_k).real / dx**3 # unitless

    return delta, delta_k, kgrid, kx, ky, kz

# 5. Top-hat filter in k-space
def W_tophat(k, R):
    x = np.where(k*R==0, 1e-30, k*R) # unitless
    return 3*(np.sin(x) - x*np.cos(x)) / x**3 # unitless

def generate_smoothed_field(R, delta_k, kgrid):
        W_R = W_tophat(kgrid, R) # unitless
        if R == 0:
            delta_k_R = delta_k # unitless
        else:
            delta_k_R = delta_k * W_R # unitless
        dx = L/N # unitless, since N is taken as Mpc in this context
        return delta_k_R, np.fft.ifftn(delta_k_R).real / dx**3  # unitless, smoothed real-space field

# 6. Compute sigma_R from delta_k
def compute_sigma_R(delta_k, kgrid, L, R):
    Volume = L**3 # Mpc^3
    W = W_tophat(kgrid, R) # unitless
    delta_k_filt = delta_k * W # unitless
    power = np.abs(delta_k_filt/Volume)**2 # ???
    var = np.sum(power) # ???
    return np.sqrt(var) # ???

# 7. Theoretical sigma_R by direct integral
def sigma_R_theory(P_fn, R):
    ks = np.logspace(-4, 1.5, 2000)
    Wks = W_tophat(ks, R)
    integrand = ks**2 * P_fn(ks) * Wks**2 / (2*np.pi**2)
    return np.sqrt(np.trapz(integrand, ks))

# 8. Run everything
R8 = 8.0 # Mpc
delta_c = 1.686 # Define the collapse threshold
dx = L / N
delta, delta_k, kgrid, kx, ky, kz = generate_gaussian_field(P_lin, N, L, seed)
sigma_sim = compute_sigma_R(delta_k, kgrid, L, R8)
sigma_th  = sigma_R_theory(P_lin, R8)
print(f"Simulated σ₈ = {sigma_sim:.4f}")
print(f"Theoretical σ₈ = {sigma_th:.4f}")

# Apply top-hat smoothing in Fourier space
slice_index = N // 2
_, delta_R8 = generate_smoothed_field(R8, delta_k, kgrid)
print(f"np.std(delta_R8) = {np.std(delta_R8):.4f}")
delta_R8_2D = delta_R8[:, :, slice_index]
print(f"np.std(delta_R8_2D) = {np.std(delta_R8_2D):.4f}")
print(f"np.std(delta) = {np.std(delta):.4f}")
print(f"np.mean(delta) = {np.mean(delta):.4f}")
"""
# Plot 2D slice of the smoothed field, including contour at delta_crit
plt.figure(figsize=(6, 5))
im = plt.imshow(delta_R8_2D.T, origin='lower', extent=[0, L, 0, L], cmap='RdBu_r')
plt.colorbar(im, label='δ (smoothed to R₈)')
cs = plt.contour(delta_R8_2D.T, levels=[delta_c], colors='k', linewidths=1.0, extent=[0, L, 0, L]) # Add contour at δ_c
plt.clabel(cs, fmt={delta_c: 'δₛ'}, inline=True, fontsize=10)
plt.title(f"2D Slice of δ Field Smoothed at R=8 Mpc (slice {slice_index})") # Labels and title
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.tight_layout()
plt.show()
""";

__Collapsing regions (projected based on linear perturbation theory)__

In [ ]:
# @title
slice_index = N // 2
delta_2D = delta[:, :, slice_index] / D_init
print("All values scaled up to a=1:")
print(f"np.std(delta) = {np.std(delta/D_init):.4f}")
print(f"np.mean(delta) = {np.mean(delta/D_init):.4f}")
print(f"delta.shape = {delta.shape}")
print(f"np.std(delta_2D) = {np.std(delta_2D):.4f}")

# Plot 2D slice of the field, including contour at delta_crit
plt.figure(figsize=(6, 5))
im = plt.imshow(delta_2D.T, origin='lower', extent=[0, L, 0, L], cmap='RdBu_r')
plt.colorbar(im, label='δ (today)')
cs = plt.contour(delta_2D.T, levels=[delta_c], colors='k', linewidths=1.0, extent=[0, L, 0, L]) # Add contour at δ_c
plt.clabel(cs, fmt={delta_c: 'δₛ'}, inline=True, fontsize=10)
plt.title(f"δ Field (projected to today)") # Labels and title
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.tight_layout()
plt.show()

__Resample delta field__

In [ ]:
# @title
delta_excerpt = delta[0:N:qg, 0:N:qg, 0:N:qg]
mean_delta_excerpt = np.mean(delta_excerpt)
print(f"np.std(delta_excerpt) = {np.std(delta_excerpt):.4f}")
print(f"np.mean(delta_excerpt) = {mean_delta_excerpt:.4f}")

# Plot 2D slice of the smoothed field, including contour at delta_crit
slice_excerpt = (N // 2) // qg
plt.figure(figsize=(6, 5))
im = plt.imshow(delta_excerpt[:,:,slice_excerpt].T, origin='lower', extent=[0, L, 0, L], cmap='RdBu_r')
plt.colorbar(im, label='δ (initial)')
cs = plt.contour(delta_excerpt[:,:,slice_excerpt].T, levels=[delta_c], colors='k', linewidths=1.0, extent=[0, L, 0, L]) # Add contour at δ_c
plt.clabel(cs, fmt={delta_c: 'δₛ'}, inline=True, fontsize=10)
plt.title(f"δ Field (initial) resampled for simulation") # Labels and title
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.tight_layout()
plt.show()

print(delta[N//2,N//2,N//2], delta_excerpt[(N//2)//qg,(N//2)//qg,(N//2)//qg])

__Create body grid and apply Zeldovich__

In [ ]:
# @title
from scipy.fft import fftn, ifftn, fftfreq
import matplotlib.pyplot as plt

# Solve Poisson equation for potential
k2 = kgrid**2
k2[0, 0, 0] = 1e-20  # zero mean potential
phi1_k = -delta_k / k2
phi1_k[0, 0, 0] = 0.0  # zero mean potential
dx_init = dx

# Compute Zel'dovich displacements and velocities
grad_phi1_x = np.fft.ifftn(-1j * kx * phi1_k).real / dx**3
grad_phi1_y = np.fft.ifftn(-1j * ky * phi1_k).real / dx**3
grad_phi1_z = np.fft.ifftn(-1j * kz * phi1_k).real / dx**3
s1 = np.stack([grad_phi1_x, grad_phi1_y, grad_phi1_z], axis=-1)  # displacement field

# --- Second-order source term: s2_k = ∇²(Φ_2) ---
# Compute second derivatives of Φ₁
phi1_xx = np.fft.ifftn(-(kx**2) * phi1_k).real
phi1_yy = np.fft.ifftn(-(ky**2) * phi1_k).real
phi1_zz = np.fft.ifftn(-(kz**2) * phi1_k).real
phi1_xy = np.fft.ifftn(-kx * ky * phi1_k).real
phi1_xz = np.fft.ifftn(-kx * kz * phi1_k).real
phi1_yz = np.fft.ifftn(-ky * kz * phi1_k).real

# Compute second-order source term S(x)
S = ( phi1_xx * phi1_yy + phi1_xx * phi1_zz + phi1_yy * phi1_zz - phi1_xy**2 - phi1_xz**2 - phi1_yz**2 )

# FFT of second-order source
S_k = np.fft.fftn(S)
phi2_k = -S_k / k2
phi2_k[0, 0, 0] = 0.0

# Second-order displacement field
grad_phi2_x = np.fft.ifftn(-1j * kx * phi2_k).real / dx**6
grad_phi2_y = np.fft.ifftn(-1j * ky * phi2_k).real / dx**6
grad_phi2_z = np.fft.ifftn(-1j * kz * phi2_k).real / dx**6
s2 = np.stack([grad_phi2_x, grad_phi2_y, grad_phi2_z], axis=-1)
spacial_ratio = np.mean(np.linalg.norm(s2, axis=1)) / np.mean(np.linalg.norm(s1, axis=1))

# Crop s1 and s2
s1_crop = s1[0:N:qp, 0:N:qp, 0:N:qp].reshape(-1, 3)
s1_crop -= np.mean(s1_crop, axis=0)
s2_crop = s2[0:N:qp, 0:N:qp, 0:N:qp].reshape(-1, 3)
s2_crop -= np.mean(s2_crop, axis=0)
s1_max = np.max(np.linalg.norm(s1_crop, axis=1))
s2_max = np.max(np.linalg.norm(s2_crop, axis=1))
D2_init = -3/7 * Ω_m**(-1/143)
f2_init = 2*f_init

# Particle positions and velocities
grid = np.indices((int(np.ceil(N/qp)), int(np.ceil(N/qp)), int(np.ceil(N/qp)))).reshape(3, -1).T #
grid_downsampled = np.all(grid % qpa == 0, axis=1) # This is for later use in resampling for animation
positions_unperturbed = grid * dx_init * qp  # unperturbed positions
test_factor = 1.0
positions = positions_unperturbed + test_factor*s1_crop  # displaced
positions += test_factor*D2_init * s2_crop # 2nd order term
velocities = test_factor*H_init * f_init * s1_crop  # in km/s
velocities += test_factor*H_init * f2_init * D2_init * s2_crop # 2nd order term
v_max = np.max(np.linalg.norm(velocities, axis=1))
v_mean = np.mean(velocities, axis=0)
velocities -= v_mean

# Display summary
print(f"Generated {positions.shape} particles")
print(f"displace.shape = {s1_crop.shape}")
print(f"positions.shape = {positions.shape}")
print(f"Initial scale factor a = {a_init:.4f}, H(a) = {H_init:.2f} km/s/Mpc")
print(f"Growth factor  D(a) = {D_init:.4f},  f(a) = {f_init:.4f}")
print(f"Max s1 dis = {s1_max:.4f} Mpc")
print(f"Max s2 dis = {D2_init * s2_max:.4f} Mpc")
print(f"Max s1 vel = {H_init * f_init * s1_max:.1f} km/s")
print(f"Max s2 vel = {H_init * f2_init * D2_init * s2_max:.1f} km/s")
print(f"Spacial ratio = {spacial_ratio:.3f}")
print(f"Max velocity = {v_max:.1f} km/s")
print(f"Mean velocity = {v_mean} km/s")

__Displacement and velocity consistency check__

In [ ]:
# @title
#s1old = s1_crop # Mpc
#s2old = s2_crop
#deltatime = (0.033/0.03-1)/(H0_cos*E(0.03)) # Myr
#deltas1 = s1_crop-s1old # Mpc
#deltas2 = (s2_crop-s2old) * D2_init/D_init
#vpec1 = deltas1/deltatime*Mpc_km/Myr_s # km/s
#vpec2 = deltas2/deltatime*Mpc_km/Myr_s # km/s
#vpec1_comp = H_init * f_init * s1_crop
#vpec2_comp = H_init * f2_init * D2_init/D_init * s2_crop

__Test of div s = -delta__

In [ ]:
# @title
def divergence_real_space(s, dx):
    """
    Compute divergence of a vector field s on a grid using central differences.
    s shape: (N, N, N, 3)
    dx: grid spacing (scalar)
    """
    div = np.zeros(s.shape[:-1])
    for i in range(3):  # x, y, z components
        ds_i = np.gradient(s[..., i], dx, axis=i)
        div += ds_i
    return div

div_s1 = divergence_real_space(s1, dx=dx)
error = div_s1 + delta

max_error = np.max(np.abs(error))
rms_error = np.sqrt(np.mean(error**2))

print(f"Max error (real-space): {max_error:.3e}")
print(f"RMS error (real-space): {rms_error:.3e}")
print(f"Max delta (real-space): {np.max(np.abs(delta)):.3e}")
print(f"RMS delta (real-space): {np.std(delta):.3e}")
print(f"Max div_s (real-space): {np.max(np.abs(div_s1)):.3e}")
print(f"RMS div_s (real-space): {np.std(div_s1):.3e}")

__Grid properties for use by n-body simulation__

In [ ]:
# @title
Ng = int(np.ceil(N/qg))  # grid size
Np = int(np.ceil(N/qp))  # body resolution in 1D
epsilon = (a_final/a_init)**(1/(steps+1))-1 # growth rate for scale factor (by iteration)
epsilon_fine = (a_final/a_init)**(1/(steps_fine+1))-1 # growth rate for scale factor (by iteration)
M_total = Ω_m * rho_crit0 * L**3  # M_sun

# Two identical regular grids for baryons and dark matter
body_positions = np.vstack([positions, positions])  # duplicate positions for baryons and dark matter
body_positions_unperturbed = np.vstack([positions_unperturbed, positions_unperturbed])
body_velocities = np.vstack([velocities, velocities])  # duplicate velocities for baryons and dark matter
grid_downsampled = np.hstack([grid_downsampled, grid_downsampled])  # duplicate for baryons and dark matter
N_bodies = body_positions.shape[0] # Number of particles
N_baryon = positions.shape[0] # TESTING
masses_baryon = np.full(N_baryon, Ω_b / Ω_m * M_total / N_baryon)
masses_dark = np.full(N_bodies - N_baryon, (Ω_m - Ω_b) / Ω_m * M_total / (N_bodies - N_baryon))
masses = np.hstack([masses_baryon, masses_dark])  # mass per particle in M_sun
baryon_set = np.hstack([np.ones(N_baryon, dtype=bool), np.zeros(N_bodies - N_baryon, dtype=bool)])
baryon_indices = np.where(baryon_set)[0]

print(f"body_positions.shape = {body_positions.shape}")
print(f"body_velocities.shape = {body_velocities.shape}")
print(f"Mean body position = {np.mean(body_positions, axis=0)} Mpc")
print(f"Mean velocity = {np.mean(body_velocities, axis=0)} km/s")

__Zeldovich Consistency Check__

In [ ]:
# @title
# Zeldovich consistency check
def show_zeldovich_consistency(delta_input, delta_measured): # Scatter plot
    correlation = np.corrcoef(delta_input, delta_measured)[0, 1]
    print(f"Correlation coefficient: {correlation:.4f}")

    plt.figure(figsize=(5, 5))
    plt.scatter(delta_input, delta_measured, s=1, alpha=0.2)
    plt.xlabel("Input δ (from field)")
    plt.ylabel("Measured δ (from displaced particles)")
    plt.title("Zeldovich Consistency Test")
    plt.plot([-1, 1], [-1, 1], 'r--')  # identity line
    plt.grid(True)
    plt.axis("equal")
    plt.show()

In [ ]:
# @title
# Produce Zeldovich Consistency Chart
dm = np.mean(delta_excerpt)
delta_input = delta_excerpt.flatten() - dm
delta_measured = grad_phi5.deposit_delta(positions=body_positions, masses=masses, Ngrid=Ng, L=L, use_TSC=use_TSC, n_threads=n_threads).flatten()
show_zeldovich_consistency(delta_input, delta_measured) # Scatter plot

__Plot initial positions and velocities__

In [ ]:
# @title
# Plot initial positions and velocities
body_z = body_positions[:, 2]
body_mask = (body_z > L/2 - qp*dx_init/2) & (body_z < L/2 + qp*dx_init/2)
vx = body_velocities[body_mask, 0]
vy = body_velocities[body_mask, 1]
x_pos = body_positions[body_mask, 0]
y_pos = body_positions[body_mask, 1]

step = 1
x_plot = x_pos[::step]
y_plot = y_pos[::step]
vx_plot = vx[::step]
vy_plot = vy[::step]

print(f"body_positions.shape = {body_positions.shape}")
print(f"body_velocities.shape = {body_velocities.shape}")
print(f"Mean body position = {np.mean(body_positions, axis=0)} Mpc")
print(f"Mean velocity = {np.mean(body_velocities, axis=0)} km/s")

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(body_positions[body_mask, 0], body_positions[body_mask, 1], s=1, alpha=0.5)
plt.title(f"Particle Distribution (z={int(1/a_init)-1}, Slice {qp*dx:.3e} Mpc)")
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.xlim(0, L)
plt.ylim(0, L)
plt.gca().set_aspect('equal')

plt.subplot(1, 2, 2)
plt.quiver(x_plot, y_plot, vx_plot, vy_plot, scale=v_max*2, width=0.002, color='darkblue')
plt.title("Velocity Field (2D slice)")
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.xlim(0, L)
plt.ylim(0, L)
plt.gca().set_aspect('equal')

plt.tight_layout()
plt.show()

# n-body simulation

In [ ]:
# @title
# Lambda tables and constants

# --- units & constants ---
Mpc_to_cm = 3.085677581e24
Myr_to_s  = 3.15576e13
Msun_to_g = 1.98847e33
m_p       = 1.6726219e-24      # g
k_B       = 1.380649e-16       # erg/K
gamma_ad  = 5.0/3.0
mu_ion    = 0.59               # fully ionized mean molecular weight
mu_neutral= 1.22               # pre-reionization
X_H       = 0.76               # H mass fraction
G_cgs     = 6.67430e-8         # cm^3 g^-1 s^-2
sigma_T   = 6.6524587158e-25     # cm^2
m_e     = 9.10938356e-28       # g
c_light = 2.99792458e10        # cm/s
a_rad   = 7.5657e-15           # erg cm^-3 K^-4

# https://local.strw.leidenuniv.nl/WSS08/
# This website hosts the Wiersma tables

# https://myhdf5.hdfgroup.org/
# This is a website that can display and manage .hdf5 files

data = np.load("cooling_table_v1.npz")
grad_phi5.set_lambda_table(data["z_grid"], data["Z_grid"], data["nH_grid"], data["T_grid"], data["Lambda"])
collis = np.load("cooling_table_collis_v1.npz")
grad_phi5.set_lambda_collis(collis["T_grid"], collis["Lambda"])

# Tiny smoke test
print("Lambda_T_nH_Z_z:")
print("Λ ~", grad_phi5.Lambda_T_nH_Z_z(1e6, 1e-3, 1.0, 0.0), "erg cm^3 s^-1") # MW-like halo gas: T~1e6 K, nH~1e-3 cm^-3, Z~1 Zsun, z=0
print("Λ ~", grad_phi5.Lambda_T_nH_Z_z(1e4, 1e-5, 0.1, 3.0), "erg cm^3 s^-1") # IGM-like: T~1e4 K, nH~1e-5 cm^-3, Z~0.1, z=3
print("Λ ~", grad_phi5.Lambda_T_nH_Z_z(1000, 1e-8, 0.0, 0.0), "erg cm^3 s^-1 (should be -6.47933756005432e-23)")

print("Lambda_collis:")
print("Λ ~", grad_phi5.Lambda_collis(1e6), "erg cm^3 s^-1")
print("Λ ~", grad_phi5.Lambda_collis(1e4), "erg cm^3 s^-1")
print("Λ ~", grad_phi5.Lambda_collis(100), "erg cm^3 s^-1")

In [ ]:
# @title
# Main thermodynamic module
from scipy.ndimage import uniform_filter
#--- density-dependent metallicity proxy: Z/Z_sun as function of (nH, z) ---
@njit
def T_CMB_of_z(z):  # K
    return 2.7255 * (1.0 + z)

@njit
def u_CMB(z, mu):
    """Specific internal energy corresponding to T_CMB(z) [erg g^-1]."""
    return 1.5 * k_B * T_CMB_of_z(z) / (mu * m_p)

@njit
def lambda_S_Compton(z, x_e=2e-4, mu=0.59, fHe=0.079, w=1.0):
    # Return (lambda, S) for the affine update du/dt = -lambda*u + S
    # representing Compton coupling. Optional weight w scales the rate.
    Tcmb = T_CMB_of_z(z)
    A = (8.0 * sigma_T * a_rad * Tcmb**4) / (3.0 * m_e * c_light)  # [s^-1]
    lam = w * A * (x_e / (1.0 + x_e + fHe))                        # [s^-1]
    S   = lam * u_CMB(z, mu)                                       # [erg g^-1 s^-1]
    return lam, S

@njit
def Zsolar_of_nH_z(
    nH,
    z,
    n0=7e-4,
    alpha=1.1,
    Z_cap=1.0,
    n_dense=1e-2,
    Z_cap_dense=3.0,
    beta_dense=0.25
):
    # nH: hydrogen number density [cm^-3]
    # z : redshift
    # returns: Z/Z_sun (dimensionless)

    # baseline redshift evolution of global metals
    Z_floor = 0.05  * 10**(-0.15 * z)   # diffuse IGM-like lower envelope
    Z_max_base   = Z_cap * 10**(-0.15 * z)   # enriched gas upper envelope
    s = (np.maximum(nH, 1e-12) / n0)**alpha

    # dense, shielded material can self-enrich beyond the diffuse cap
    dense_ratio = np.maximum(nH / np.maximum(n_dense, 1e-30), 1.0)
    dense_boost = np.minimum(dense_ratio**beta_dense, Z_cap_dense / np.maximum(Z_cap, 1e-30))
    Z_max_dense = Z_max_base * dense_boost

    Z_target = np.minimum(Z_max_dense, Z_cap_dense * 10**(-0.1 * z))
    return Z_floor + (Z_target - Z_floor) * (s / (1.0 + s))

def x_e_residual(z):
    # Residual free electron fraction (very approximate; clamp to sane bounds)
    # ~2e-4 at z~100 with mild z dependence
    xe = 2.0e-4 * ((1.0 + z)/100.0)**0.5
    return np.clip(xe, 1e-5, 3e-4)

# temperature/internal-energy conversions ----
@njit
def u_from_T(T, mu=mu_ion, gamma=gamma_ad):
    return (k_B * T) / ((gamma - 1.0) * mu * m_p) # erg/g

@njit
def T_from_u(u, mu=mu_ion, gamma=gamma_ad):
    return (gamma - 1.0) * u * mu * m_p / k_B # K

@njit
def shielding_factor_totalH(
    nH,                 # hydrogen number density [cm^-3]
    rho_b_phys,         # total physical mass density in the cell [g cm^-3]
    dx_phys_cm,         # physical cell size [cm]
    mean_baryon_mass,   # mean baryon mass [g]
    Zsol,               # metallicity relative to solar
    alpha_len=2.0,      # multiplier for safety margin on L_sh
    sigma_eff=2.0e-18,  # effective gas cross section [cm^2]
    sigma_d=1.0e-21     # dust cross section per H per Z' [cm^2]
):
    # Computes a per-cell total self-shielding factor f_sh (0–1)
    # using total hydrogen and a characteristic local shielding length.

    tiny = 1e-40 #    Avoid division by zero where rho = 0
    ell_p = np.power(np.maximum(mean_baryon_mass / np.maximum(rho_b_phys, tiny), 0.0), 1.0/3.0) # Inter-particle spacing per cell (in cm)
    L_sh = alpha_len * np.maximum(dx_phys_cm, ell_p) # Local shielding length
    NH = nH * L_sh # Total hydrogen column [cm^-2]
    f_gas  = np.exp(-sigma_eff * NH) # Gas and dust attenuation
    f_dust = np.exp(-sigma_d * Zsol * NH)
    f_sh = f_gas * f_dust # Combined self-shielding factor
    return f_sh

@njit
def jeans_temperature_floor(rho_phys, dx_phys_cm, mu, n_jeans, gamma=gamma_ad, G_cgs=G_cgs):
    """Return a temperature floor so the Jeans length ≥ n_jeans * Δx."""
    if n_jeans <= 0: return np.zeros_like(rho_phys)
    coeff = (mu * m_p) / (gamma * k_B)
    factor = (n_jeans**2 * G_cgs) / np.pi
    return coeff * factor * rho_phys * (dx_phys_cm**2)

@njit
def binned_means(values, bins, bin_values):
    # Compute the mean of 'values' in bins defined by 'bins', based on 'bin_values'.
    means = []
    for i in range(len(bins) - 1):
        mask = (bin_values >= bins[i]) & (bin_values < bins[i + 1])
        if np.any(mask): means.append(float(np.mean(values[mask])))
        else: means.append(np.nan)
    return means

def cooling_heating_step_baryons_py(
    pos_baryon, vel_baryon, masses_baryon, rho_b_coarse_com_mean,
    U_baryon, Lbox_Mpc, Ng, dt_Myr, a,
    use_TSC = False, n_threads=8, z_reion=8.0, dz_trans=0.5, C_cfl=0.2, smooth = 1
):
    global n_jeans
    # Mesh-based thermochemistry:
    #  1) deposit baryon mass -> rho_b_com_grid
    #  2) deposit particle u -> u_grid (mass-weighted == number-weighted for equal-mass)
    #  3) compute cell-wise nH, T, rates; update u_grid with limiter
    #  4) gather updated u back to baryon particles
    # Returns: u_new (N,), T_part (on baryons), diag dict

    z_now = 1.0/np.maximum(a, 1e-6) - 1.0
    dt_s  = dt_Myr * Myr_to_s # s

    # Densities on grid: comoving, then physical, then nH
    rho_b_com_grid, dx = grad_phi5.deposit_density(pos_Mpc=pos_baryon, masses=masses_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # Mpc^-3
    if smooth > 1: rho_b_com_grid = uniform_filter(rho_b_com_grid, size=smooth, mode='nearest')
    od_b_grid = rho_b_com_grid / rho_b_coarse_com_mean # unitless
    max_b_od = np.max(od_b_grid) # unitless
    rho_b_com_grid *=  Msun_to_g / Mpc_to_cm**3 # g/cm^3
    rho_b_phys_grid = rho_b_com_grid / a**3 # g/cm^3
    nH_grid       = (X_H * rho_b_phys_grid) / m_p # cm^-3
    nH_mean   = float(np.mean(nH_grid)) # for UV calibration
    nH_max    = float(np.max(nH_grid)) # for UV calibration
    w_hi = 1.0 / (1.0 + np.exp(-(z_now - z_reion)/dz_trans)) # unitless
    w_reion = 1.0 - w_hi

    Z_sol = Zsolar_of_nH_z(nH_grid, z_now)
    mean_baryon_mass = np.mean(masses_baryon) # Msun

    f_sh = shielding_factor_totalH(
        nH=nH_grid,
        rho_b_phys=rho_b_phys_grid,
        dx_phys_cm=dx * a * Mpc_to_cm,
        mean_baryon_mass=mean_baryon_mass * Msun_to_g,
        Zsol=Z_sol
    )
    shield_weight = np.minimum(np.maximum(1.0 - f_sh, 0.0), 1.0)
    mu_grid = (
        mu_neutral * (w_hi + shield_weight * w_reion) +
        mu_ion * ((1.0 - shield_weight) * w_reion)
    )

    # Deposit u to grid (erg/g), compute T on grid with μ(a)
    U_grid, _ = grad_phi5.deposit_scalar(pos_Mpc=pos_baryon, masses=masses_baryon, scalar=U_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # erg/g
    if smooth > 1: U_grid = uniform_filter(U_grid, size=smooth, mode='nearest')
    T_jeans = jeans_temperature_floor(rho_b_phys_grid, dx*a*Mpc_to_cm, mu_grid, n_jeans=n_jeans, gamma=gamma_ad, G_cgs=G_cgs)
    T_grid = np.clip(T_from_u(U_grid, mu_grid), np.maximum(T_CMB_of_z(z_now), T_jeans), 1e9) # K
    U_grid = u_from_T(T_grid, mu_grid) # re-apply clipped T_grid to derive U_grid, erg g^-1

    T_jeans_min = np.min(T_jeans)
    T_jeans_max = np.max(T_jeans)
    T_jeans_mean = np.mean(T_jeans)
    T_jeans_mwm = np.mean(T_jeans*rho_b_phys_grid)/np.mean(rho_b_phys_grid)

    # Pressure term, including artificial viscosity
    vx, vy, vz = grad_phi5.deposit_vector(pos_Mpc=pos_baryon, masses=masses_baryon, vec=vel_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # Mpc/Myr
    vx *= Mpc_to_cm / Myr_to_s; vy *= Mpc_to_cm / Myr_to_s; vz *= Mpc_to_cm / Myr_to_s # cm/s
    P_phys_grid = (gamma_ad - 1.0) * rho_b_phys_grid * U_grid # erg/cm^3
    q_visc, div_v_s, S_min = grad_phi5.artificial_viscosity_q_cgs( # erg/cm^3, 1/s, unitless
        rho_b_phys_grid,
        U_grid,
        vx,
        vy,
        vz,
        dx*Mpc_to_cm,
        a,
        C2=0.5,
        C1=0.0,
        Ctheta = 5.0
    )
    q_visc = np.where(div_v_s < 0.0, q_visc, 0.0)
    P_phys_grid += q_visc  # use effective pressure everywhere, erg/cm^3

    # pressure acceleration on mesh, then gather to baryons ---
    aPx, aPy, aPz    = grad_phi5.pressure_acceleration_grid(rho_b_com_grid=rho_b_com_grid, P_phys=P_phys_grid, a=a, dx=dx*Mpc_to_cm) # erg cm^-1 g^-1 = cm s^-2
    aPx *= Myr_to_s**2 / Mpc_to_cm # Mpc/Myr^2
    aPy *= Myr_to_s**2 / Mpc_to_cm
    aPz *= Myr_to_s**2 / Mpc_to_cm
    aP_new = grad_phi5.gather_vector(ax=aPx, ay=aPy, az=aPz, pos_Mpc=pos_baryon, Ngrid=Ng, Lbox_Mpc=Lbox_Mpc, use_TSC=use_TSC, n_threads=n_threads) # Mpc/Myr^2

    # CFL (Courant–Friedrichs–Lewy) timestep ---
    # Sound speed from u: c_s^2 = γ(γ-1) u  (u in erg/g => cm^2/s^2)
    c_s_cms_grid = np.sqrt(np.maximum(gamma_ad * (gamma_ad - 1.0) * U_grid, 0.0))
    c_s_max_cms  = float(np.max(c_s_cms_grid))
    v_max_Mpc_Myr = float(np.max(np.linalg.norm(vel_baryon, axis=1))) # Combine with max baryon speed (particles)
    c_s_max_Mpc_Myr = c_s_max_cms * (Myr_to_s / Mpc_to_cm) / max(a, 1e-12)
    dt_cfl = C_cfl * dx / max(c_s_max_Mpc_Myr + v_max_Mpc_Myr, 1e-20)

    # Transition weight (scalar): 1 at high-z (primordial branch), 0 at low-z
    w_thres = 1e-5 # unitless
    epochs = [w_hi > w_thres, 1.0 - w_hi > w_thres] # boolean

    lam_shielded = np.zeros_like(U_grid)
    S_shielded   =  np.zeros_like(U_grid)
    lam_unshielded = np.zeros_like(U_grid)
    S_unshielded   =  np.zeros_like(U_grid)

    x_e_floor = x_e_residual(z_now)
    x_e_grid = x_e_floor + (1.0 - x_e_floor) * (1.0 - shield_weight) * w_reion
    x_e_grid = np.minimum(np.maximum(x_e_grid, x_e_floor), 1.0)

    Zsol_min = np.min(Z_sol)
    Zsol_max = np.max(Z_sol)
    Zsol_mean = np.mean(Z_sol)

    C_CIE = (nH_grid**2) * grad_phi5.Lambda_collis(T=T_grid, n_threads=n_threads) # erg cm^-3 s^-1
    C_PIE = (nH_grid**2) * grad_phi5.Lambda_T_nH_Z_z(T=T_grid, nH=nH_grid, Z_solar=Z_sol, z=z_now, n_threads=n_threads) # erg cm^-3 s^-1
    lam_comp, S_comp = lambda_S_Compton(z_now, x_e=x_e_grid, mu=mu_grid) # s^-1, erg g^-1 s^-1

    # Cell-wise cooling/heating rates
    if epochs[0]: # --- High-z: primordial, Unshielded = Total = CIE+Compton, Shielded = 0
        du_dt_cool_grid = C_CIE / np.maximum(rho_b_phys_grid, 1e-32) # erg g^-1 s^-1
        lam_unshielded += w_hi / U_grid * du_dt_cool_grid # Net Cooling/Heating, s^-1
        lam_unshielded += w_hi * lam_comp # s^-1
        S_unshielded   += w_hi * S_comp # Compton, erg g^-1 s^-1

    if epochs[1]: # --- Low-z: Shielded = PIE-CIE-Compton, Unshielded = +CIE+Compton, Total = Unshielded + f_sh * Shielded
        du_dt_cool_grid = (C_PIE - C_CIE) / np.maximum(rho_b_phys_grid, 1e-32) # erg g^-1 s^-1
        lam_shielded += (1.0 - w_hi) / U_grid * du_dt_cool_grid # Heating only, s^-1
        lam_shielded -= (1.0 - w_hi) * lam_comp # s^-1
        S_shielded   -= (1.0 - w_hi) * S_comp # Compton, erg g^-1 s^-1

        du_dt_cool_grid = (C_CIE) / np.maximum(rho_b_phys_grid, 1e-32) # erg g^-1 s^-1
        lam_unshielded += (1.0 - w_hi) / U_grid * du_dt_cool_grid # Cooling only, s^-1
        lam_unshielded += (1.0 - w_hi) * lam_comp # s^-1
        S_unshielded   += (1.0 - w_hi) * S_comp # Compton, erg g^-1 s^-1

    lam_unshielded += (gamma_ad - 1.0) * (div_v_s / a) # pdv, s^-1
    S_unshielded   += -q_visc / np.maximum(rho_b_phys_grid, 1e-40) * (div_v_s / a) # Artificial Viscosity, erg g^-1 s^-1

    f_sh_min = np.min(f_sh)
    f_sh_max = np.max(f_sh)
    f_sh_mean = np.mean(f_sh)
    f_sh_mwm = np.mean(f_sh*rho_b_phys_grid)/np.mean(rho_b_phys_grid)

    lam = lam_unshielded + f_sh * lam_shielded
    S = S_unshielded + f_sh * S_shielded

    # Solve: du/dt = -λ u + S
    mask = np.abs(lam*dt_s) > 1e-10 # Thats when exponential update works
    e = np.exp(-lam * dt_s) # unitless
    U_grid_new = np.zeros_like(U_grid)
    U_grid_new[mask] = U_grid[mask] * e[mask] + (S[mask] / lam[mask]) * (1.0 - e[mask]) # exponential update, erg g^-1
    U_grid_new[~mask] = U_grid[~mask] + S[~mask]*dt_s # linear update, erg g^-1
    H_a = H0_cos * E(a) # 1/Myr
    U_grid_new += -2.0 * dt_Myr * H_a * U_grid # Hubble component, # erg g^-1
    T_grid_new = np.clip(T_from_u(U_grid_new, mu_grid), np.maximum(T_CMB_of_z(z_now), T_jeans), 1e9) # K
    U_grid_new = u_from_T(T_grid_new, mu_grid) # re-apply clipped T_grid_new to derive U_grid, erg g^-1

    # Gather updated u back to particles; also return particle T and P for convenience
    U_new = grad_phi5.gather_scalar(grid=U_grid_new, pos_Mpc=pos_baryon, Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads) # erg g^-1
    T_new = grad_phi5.gather_scalar(grid=T_grid_new, pos_Mpc=pos_baryon, Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads) # K
    P_new = grad_phi5.gather_scalar(grid=P_phys_grid, pos_Mpc=pos_baryon, Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads)

    od_bins = [0.0, 1.0, 10.0, 100.0, 1000.0, np.inf] # Overdensity bins: [0–1), [1–10), [10–100), [100–1000), [1000+)
    temp_by_b_od = binned_means(T_grid.ravel(), od_bins, od_b_grid.ravel() ) # Compute the mean temperatures in each overdensity bin

    diagnosis = dict(
        nH_mean = nH_mean,
        nH_max = nH_max,
        Zsol_min = Zsol_min,
        Zsol_max = Zsol_max,
        Zsol_mean = Zsol_mean,
        f_sh_min = f_sh_min,
        f_sh_max = f_sh_max,
        f_sh_mean = f_sh_mean,
        f_sh_mwm = f_sh_mwm,
        T_jeans_min = T_jeans_min,
        T_jeans_max = T_jeans_max,
        T_jeans_mean = T_jeans_mean,
        T_jeans_mwm = T_jeans_mwm
    )
    return U_new, T_new, aP_new, P_new, diagnosis, epochs, dt_cfl, max_b_od, np.nan, temp_by_b_od, S_min

# Initialize internal energy u at a_init (z_init)
def init_internal_energy(pos_Mpc, baryon_mask, masses, Lbox_Mpc, Ng, a_init,
                         model="pre_reion_adiabatic", T_floor=50.0,
                         z_dec=150.0, n_threads=8):
    # Initialize u [erg/g] at a_init.
    # - pre_reion_adiabatic: T starts from T_CMB(z_dec) and scales ∝ (1+z)^2, with optional mild density boost.
    # - post_reion_uniform : T = 1e4 K everywhere.

    # baryon density field (for optional compressional boost)
    z_init = 1.0/np.maximum(a_init, 1e-6) - 1.0
    rho_b_com_grid, dx = grad_phi5.deposit_density(pos_Mpc=pos_Mpc[baryon_mask], masses=masses[baryon_mask], Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # Mpc^-3
    rho_b_com_grid *= Msun_to_g / Mpc_to_cm**3
    rho_phys_part = grad_phi5.gather_scalar(grid=rho_b_com_grid / a_init**3, pos_Mpc=pos_Mpc[baryon_mask], Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads)
    rho_phys_mean = np.mean(rho_phys_part)

    if model == "pre_reion_adiabatic":
        # Start from CMB at decoupling, then adiabatic T ∝ a^-2
        T_dec = T_CMB_of_z(z_dec)
        T_ad  = T_dec * ((1.0 + z_init) / (1.0 + z_dec))**2
        # Never below CMB at the start
        T_gas = np.maximum(T_ad, T_CMB_of_z(z_init))
        # Optional mild compressional boost (use small exponent to avoid bias from particle sampling)
        beta = (gamma_ad - 1.0) * 0.3  # e.g. 30% of the full exponent
        T_gas *= np.power(np.maximum(rho_phys_part / np.maximum(rho_phys_mean, 1e-99), 1e-3), beta)
        T_gas = np.maximum(T_gas, T_floor)
        mu = mu_neutral  # neutral before reionization
    elif model == "post_reion_uniform":
        T_gas = np.full(np.count_nonzero(baryon_mask), 1.0e4)
        mu = mu_ion
    else:
        raise ValueError("Unknown init model")

    return u_from_T(T_gas, mu=mu, gamma=gamma_ad)


def temp_pressure_py(
    pos_baryon, vel_baryon, masses_baryon, rho_b_coarse_com_mean,
    U_baryon, Lbox_Mpc, Ng, a,
    use_TSC = False, n_threads=8, z_reion=8.0, dz_trans=0.5, smooth = 1
):
    z_now = 1.0/np.maximum(a, 1e-6) - 1.0

    # Densities on grid: comoving, then physical, then nH
    rho_b_com_grid, dx = grad_phi5.deposit_density(pos_Mpc=pos_baryon, masses=masses_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # Mpc^-3
    if smooth > 1: rho_b_com_grid = uniform_filter(rho_b_com_grid, size=smooth, mode='nearest')
    od_b_grid = rho_b_com_grid / rho_b_coarse_com_mean # unitless
    rho_b_com_grid *=  Msun_to_g / Mpc_to_cm**3 # g/cm^3
    rho_b_phys_grid = rho_b_com_grid / a**3 # g/cm^3
    nH_grid       = (X_H * rho_b_phys_grid) / m_p # cm^-3
    w_hi = 1.0 / (1.0 + np.exp(-(z_now - z_reion)/dz_trans)) # unitless
    w_reion = 1.0 - w_hi

    Z_sol = Zsolar_of_nH_z(nH_grid, z_now)
    mean_baryon_mass = np.mean(masses_baryon) # Msun

    f_sh = shielding_factor_totalH(
        nH=nH_grid,
        rho_b_phys=rho_b_phys_grid,
        dx_phys_cm=dx * a * Mpc_to_cm,
        m_particle=mean_baryon_mass * Msun_to_g,
        Zsol=Z_sol
    )
    shield_weight = np.minimum(np.maximum(1.0 - f_sh, 0.0), 1.0)
    mu_grid = (
        mu_neutral * (w_hi + shield_weight * w_reion) +
        mu_ion * ((1.0 - shield_weight) * w_reion)
    )

    # Deposit u to grid (erg/g), compute T on grid with μ(a)
    U_grid, _ = grad_phi5.deposit_scalar(pos_Mpc=pos_baryon, masses=masses_baryon, scalar=U_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # erg/g
    if smooth > 1: U_grid = uniform_filter(U_grid, size=smooth, mode='nearest')
    T_grid = T_from_u(U_grid, mu_grid) # K

    # Pressure term, including artificial viscosity
    vx, vy, vz = grad_phi5.deposit_vector(pos_Mpc=pos_baryon, masses=masses_baryon, vec=vel_baryon, Lbox_Mpc=Lbox_Mpc, Ng=Ng, use_TSC=use_TSC) # Mpc/Myr
    vx *= Mpc_to_cm / Myr_to_s; vy *= Mpc_to_cm / Myr_to_s; vz *= Mpc_to_cm / Myr_to_s # cm/s
    P_phys_grid = (gamma_ad - 1.0) * rho_b_phys_grid * U_grid # erg/cm^3
    q_visc, div_v_s, S_min = grad_phi5.artificial_viscosity_q_cgs( # erg/cm^3, 1/s, unitless
        rho_b_phys_grid,
        U_grid,
        vx,
        vy,
        vz,
        dx*Mpc_to_cm,
        a,
        C2=0.5,
        C1=0.0,
        Ctheta = 5.0
    )
    q_visc = np.where(div_v_s < 0.0, q_visc, 0.0)
    P_phys_grid += q_visc  # use effective pressure everywhere, erg/cm^3

    # Gather updated u back to particles; also return particle T and P for convenience
    T_new = grad_phi5.gather_scalar(grid=T_grid, pos_Mpc=pos_baryon, Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads) # K
    P_new = grad_phi5.gather_scalar(grid=P_phys_grid, pos_Mpc=pos_baryon, Lbox_Mpc=Lbox_Mpc, dx=dx, use_TSC=use_TSC, n_threads=n_threads)

    od_bins = [0.0, 1.0, 10.0, 100.0, 1000.0, np.inf] # Overdensity bins: [0–1), [1–10), [10–100), [100–1000), [1000+)
    temp_by_b_od = binned_means(T_grid.ravel(), od_bins, od_b_grid.ravel() ) # Compute the mean temperatures in each overdensity bin

    return T_new, P_new, temp_by_b_od

In [ ]:
# @title
# Gravity mesh refinement
from numpy.fft import fftn, ifftn
from scipy.fftpack import dst, idst   # dst/type=1 diagonalizes Dirichlet Laplacian
import math

offset = 0.5

@njit
def laplace(phi, dx):
    return (phi[2:  ,1:-1,1:-1] - 2.0*phi[1:-1,1:-1,1:-1] + phi[0:-2,1:-1,1:-1]) / dx**2 + (phi[1:-1,2:  ,1:-1] - 2.0*phi[1:-1,1:-1,1:-1] + phi[1:-1,0:-2,1:-1]) / dx**2 + (phi[1:-1,1:-1,2:  ] - 2.0*phi[1:-1,1:-1,1:-1] + phi[1:-1,1:-1,0:-2]) / dx**2

# --- Solve Poisson on fine grid using Discrete Sine Transform
def refine_subgrid(phi_fine, rho_fine_com_interior, rho_bar, Nf, dx_fine, a, G, eps_eig=1e-15):
    # Safe wrapper around DST-based fine-grid Poisson solve.
    # - phi_fine: full array including ghost cells (so laplace(phi_fine,dx_fine) returns shape (Nf,Nf,Nf))
    # - rho_fine_com_interior: shape (Nf,Nf,Nf) (rho per cell)
    # - rho_bar: mean density of entire grid (not just the fine box). Usually matches cosmic mean
    # - Nf: interior size per axis
    # - dx_fine: fine grid spacing (scalar > 0)
    # - G: gravitational constant in code units
    # - eps_eig: tiny threshold to detect tiny eigenvalues
    # Returns psi_interior (shape (Nf,Nf,Nf)) solving Δψ = rhs with ψ=0 on boundary (you must add phi back)

    # basic checks
    if dx_fine <= 0.0: raise ValueError("dx_fine must be positive")
    if not isinstance(Nf, int) or Nf <= 0: raise ValueError("Nf must be a positive integer")
    if rho_fine_com_interior.shape != (Nf, Nf, Nf): raise ValueError(f"rho_fine_com_interior shape {rho_fine_com_interior.shape} does not match (Nf,Nf,Nf)=({Nf},{Nf},{Nf})")
    # compute Lap(P)
    lapP_full = laplace(phi_fine, dx_fine)    # MUST return (Nf,Nf,Nf) interior

    if lapP_full.shape != (Nf, Nf, Nf): raise ValueError("laplace(...) did not return expected interior shape (Nf,Nf,Nf)")
    # build rhs and check finiteness
    rhs = 4.0 * math.pi * G * (rho_fine_com_interior - rho_bar)/a - lapP_full
    if not np.isfinite(rhs).all(): raise ValueError("rhs contains NaN or Inf")

    # forward DST-I along axes (scipy.fftpack.dst type=1). Ensure contiguous float64 arrays for speed
    rhs = np.asarray(rhs, dtype=np.float64, order='C')
    fhat = dst(dst(dst(rhs, type=1, axis=0), type=1, axis=1), type=1, axis=2)

    # eigenvalues for Dirichlet Laplacian (k=1..Nf)
    k = np.arange(1, Nf + 1)
    lam1d = 2.0 * (np.cos(np.pi * k / (Nf + 1.0)) - 1.0) / (dx_fine**2)
    # lam1d should be strictly negative; form lam3d by broadcasting
    lam3d = lam1d[:, None, None] + lam1d[None, :, None] + lam1d[None, None, :]

    # safety: check no tiny eigenvalues (shouldn't happen for k=1..Nf)
    lam_min = np.abs(lam3d).min()
    if lam_min < eps_eig: raise RuntimeError(f"Very small Laplacian eigenvalue detected (min abs {lam_min:.3e}) - check dx_fine/Nf")

    # solve in spectral space
    psihat = fhat / lam3d   # safe because lam3d != 0

    # inverse triple DST-I
    psi = idst(idst(idst(psihat, type=1, axis=2), type=1, axis=1), type=1, axis=0)

    # normalization factor that matches scipy.fftpack.dst/idst (un-normalized DST-I)
    norm = (2.0 * (Nf + 1.0))**3
    return psi / norm   # shape (Nf,Nf,Nf)

def create_subgrid(cube_center, cube_size, r, Lc, Nc, pos_all, baryon_mask, phi_coarse, margin=3.5, use_TSC=False):
    dx = Lc/Nc

    # define refined grid aligned to coarse cells (choose integer cell indices)
    # choose coarse indices that define cube: find range of coarse cells covering cube
    i_min = int(((cube_center[0]-cube_size/2.0) / Lc) * Nc+1)
    j_min = int(((cube_center[1]-cube_size/2.0) / Lc) * Nc+1)
    k_min = int(((cube_center[2]-cube_size/2.0) / Lc) * Nc+1)
    n_cells = int((cube_size / Lc) * Nc)  # number of coarse cells per side

    # Refine grid
    Nf = n_cells * r
    box_min_fine = np.array([i_min*dx, j_min*dx, k_min*dx]) # fine box min/max in physical coords
    box_max_fine = box_min_fine + n_cells*dx
    dx_fine = dx / r
    L_fine = Nf * dx_fine

    # Mask particles within fine box and deposit to fine rho grid
    mask_inside = np.all((pos_all >= box_min_fine) & (pos_all < box_max_fine), axis=1)
    mask_interior = np.all((pos_all >= box_min_fine + margin*dx_fine) & (pos_all < box_max_fine - margin*dx_fine), axis=1)

    # Mask particles within fine box and deposit to fine rho grid
    pos_b = pos_all[baryon_mask]
    mask_b_inside = np.all((pos_b >= box_min_fine) & (pos_b < box_max_fine), axis=1)
    mask_b_interior = np.all((pos_b >= box_min_fine + margin*dx_fine) & (pos_b < box_max_fine - margin*dx_fine), axis=1)

    if mask_inside.sum() == 0:
        print("No high-res particles found inside cube! adjust cube.")
        return

    # --- Interpolate coarse phi onto fine phi (incl ghost) as initial guess ---
    # build coordinates of fine interior cell centers (shape Nf)
    xs = (np.arange(-1, Nf+1) + offset) * dx_fine + box_min_fine[0]
    ys = (np.arange(-1, Nf+1) + offset) * dx_fine + box_min_fine[1]
    zs = (np.arange(-1, Nf+1) + offset) * dx_fine + box_min_fine[2]

    # create flattened list of points (M = Nf^3, shape (M,3))
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing='ij')
    pts = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

    # evaluate coarse phi at these fine points using trilinear_interpolate helper
    vals = grad_phi5.gather_scalar(grid=phi_coarse, pos_Mpc=pts, Lbox_Mpc=Lc, dx=dx, use_TSC=use_TSC, n_threads=n_threads)
    phi_fine_est = vals.reshape((Nf+2, Nf+2, Nf+2)) # Create phi_fine (reshape to grid)

    return box_min_fine, box_max_fine, L_fine, dx_fine, Nf, phi_fine_est, mask_inside, mask_interior, mask_b_inside, mask_b_interior

def print_subgrid_parameters(cube_size, r, Lc, Nc, margin, file):
    if cube_size is None or r is None: return

    dx = Lc/Nc
    n_cells = int((cube_size / Lc) * Nc)  # number of coarse cells per side
    Nf = n_cells * r
    dx_fine = dx / r
    L_fine = Nf * dx_fine
    print(f"Fine mesh: dx_fine={dx_fine:.3f} Mpc, L_fine={L_fine:.1f} Mpc, Nc_fine={n_cells}, Nf_fine={Nf}, Nf_interior={Nf-2*int(margin)}", file=file)

def create_rho_fine(cube_center, cube_size, r, Lc, Nc, pos_all, masses_all, use_TSC):
    if cube_size is None or cube_center is None or r is None: return None

    # define refined grid aligned to coarse cells (choose integer cell indices)
    i_min = int(((cube_center[0]-cube_size/2.0) / Lc) * Nc+1)
    j_min = int(((cube_center[1]-cube_size/2.0) / Lc) * Nc+1)
    k_min = int(((cube_center[2]-cube_size/2.0) / Lc) * Nc+1)
    n_cells = int((cube_size / Lc) * Nc)  # number of coarse cells per side

    # Refine grid
    dx = Lc/Nc
    Nf = n_cells * r
    box_min_fine = np.array([i_min*dx, j_min*dx, k_min*dx]) # fine box min/max in physical coords
    box_max_fine = box_min_fine + n_cells*dx
    dx_fine = dx / r
    L_fine = Nf * dx_fine

    mask_inside = np.all((pos_all >= box_min_fine) & (pos_all < box_max_fine), axis=1)
    rho_fine_com_interior, _ = grad_phi5.deposit_density(
        pos_Mpc = pos_all[mask_inside] - box_min_fine, # moved to origin
        masses = masses_all[mask_inside],
        Lbox_Mpc = L_fine,
        Ng = Nf,
        use_TSC = use_TSC
    )
    return rho_fine_com_interior # Msun/Mpc^3

def refine_potential(x_inside_rel, phi_fine_est, dx_fine, Nf, masses_inside, rho_coarse_com_mean, a, G, use_TSC, n_threads):
    rho_fine_com_interior, _ = grad_phi5.deposit_density(
        pos_Mpc = x_inside_rel, # new positions, moved to origin
        masses = masses_inside,
        Lbox_Mpc = dx_fine*Nf,
        Ng = Nf,
        use_TSC = use_TSC
    )
    phi_local = phi_fine_est.copy()
    phi_local[1:-1,1:-1,1:-1] += grad_phi5.refine_subgrid(
        phi_fine=phi_local,
        rho_fine_com_interior=rho_fine_com_interior,
        rho_bar=rho_coarse_com_mean,
        Nf=Nf,
        dx_fine=dx_fine,
        a=a,
        G=G,
        n_threads=n_threads
    )
    ax, ay, az = grad_phi5.gradient_central(grid=phi_local, dx=dx_fine)
    acc_inside = grad_phi5.gather_vector(ax=-ax[1:-1,1:-1,1:-1], ay=-ay[1:-1,1:-1,1:-1], az=-az[1:-1,1:-1,1:-1], pos_Mpc=x_inside_rel, Ngrid=Nf, Lbox_Mpc=Nf*dx_fine, use_TSC=use_TSC, n_threads=n_threads)
    return acc_inside, np.max(rho_fine_com_interior)/rho_coarse_com_mean

In [ ]:
# @title
# Combined step (leapfrog gravity and thermo)

from concurrent.futures import ThreadPoolExecutor

def combined_step_parallel_py(x, u, g, gP_b, U_b, baryon_mask, masses, total_mass, L, Ng, dt, a, G, eng, use_TSC, n_threads, f_dyn=0.2):
    global cube_fine, center_fine, refinement, od_threshold, f_smooth, subcycles, margin

    H_n   = H0_cos * E(a) # at a^n
    a_half = a + 0.5*dt * a * H_n # predictor for mid-time scale factor
    H_half = H0_cos * E(a_half)
    u_half = u + 0.5 * dt * (g - 2 * H_half * u)
    u_half[baryon_mask] += 0.5 * dt * gP_b
    x_new = (x + dt * u_half) % L
    a_new = a + dt * a * H_half
    rho_b_coarse_com_mean = Ω_b/Ω_m*total_mass/L**3

    with ThreadPoolExecutor(max_workers=2) as executor:
        # Run gravity solve and thermo update concurrently (both release the GIL).
        future_eng = executor.submit(
            eng.step,
            pos=x_new,
            masses=masses,
            total_mass=total_mass,
            a=a_new,
            G=G
        )
        future_cooling = executor.submit(
            cooling_heating_step_baryons_py, #grad_phi5.cooling_heating_step
            pos_baryon=x_new[baryon_mask],
            vel_baryon=u_half[baryon_mask],
            masses_baryon=masses[baryon_mask],
            rho_b_coarse_com_mean=rho_b_coarse_com_mean,
            U_baryon=U_b,
            Lbox_Mpc=L,
            Ng=Ng,
            dt_Myr=dt,
            a=a_new,
            use_TSC=use_TSC,
            n_threads=n_threads,
            smooth=0 # coarse grid, no smoothing
        )

        g_new, phi_coarse, PE, diagnosis2 = future_eng.result()
        U_b_new, T_b_new, gP_b_new, P_b_new, diagnosis1, epochs, dt_target_new, max_b_od, _, temp_by_b_od, S_max = future_cooling.result()

    u_new = u_half + 0.5 * dt * (g_new - 2 * H_half * u_half)
    u_new[baryon_mask] += 0.5 * dt * gP_b_new

    max_tot_od = diagnosis2.get('max_tot_od', 0.0)
    dt_dyn = diagnosis2.get('dt_dyn', 1e100) # Extract dt_dyn from gravity step
    dt_target_new = min(dt_target_new, dt_dyn) # determine new dt_target

    cube_fine_active = cube_fine if max_tot_od > od_threshold else None # Mpc
    if cube_fine_active is not None and center_fine is not None and refinement is not None and subcycles > 0:
        rho_coarse_com_mean = total_mass/L**3 # Msun/Mpc^3

        box_min_fine, box_max_fine, L_fine, dx_fine, Nf, phi_fine_est, mask_inside, mask_interior, mask_b_inside, mask_b_interior = create_subgrid( # create the subgrid
            cube_center = center_fine,
            cube_size = cube_fine_active,
            r = refinement,
            Lc = L,
            Nc = Ng,
            pos_all = x, # all bodies, before drift
            baryon_mask = baryon_mask,
            phi_coarse = phi_coarse,
            margin = margin,
            use_TSC = use_TSC
        )

        x_inside = x[mask_inside].copy() # pos all bodies inside fine cube
        masses_inside = masses[mask_inside].copy() # mass all bodies inside fine cube
        u_inside = u[mask_inside].copy() # vel all bodies inside fine cube
        g_inside = g[mask_inside].copy() # acc all bodies inside fine cube
        U_b_inside = U_b[mask_b_inside].copy() # Energy baryons inside fine cube
        gP_b_inside = gP_b[mask_b_inside].copy() # pressure acceleration baryons inside fine cube
        a_inside = a # initial scale factor for subcycling
        dt_inside = dt / subcycles # time step for subcycling
        baryon_mask_inside = baryon_mask[mask_inside]  # apply to baryon
        Zsol_max_fine = 0
        nH_max_fine = 0

        # LOOP SUBCYCLES
        for i in range(subcycles):
            H_inside   = H0_cos * E(a_inside) # at a^n
            a_inside_half = a_inside + 0.5*dt_inside * a_inside * H_inside # predictor for mid-time scale factor
            H_inside_half = H0_cos * E(a_inside_half)
            u_inside_half = u_inside + 0.5 * dt_inside * (g_inside - 2 * H_inside_half * u_inside)
            u_inside_half[baryon_mask_inside] += 0.5 * dt_inside * gP_b_inside
            x_inside_new = x_inside + dt_inside * u_inside_half
            a_inside_new = a_inside + dt_inside * a_inside * H_inside_half

            with ThreadPoolExecutor(max_workers=2) as executor:
                # Parallelize fine-grid gravity solve with baryon cooling at this sub-step.
                future_phi = executor.submit(
                    refine_potential, #grad_phi5.refine_potential
                    x_inside_rel = x_inside_new - box_min_fine, # moved to origin
                    phi_fine_est = phi_fine_est,
                    dx_fine = dx_fine,
                    Nf = Nf,
                    masses_inside = masses_inside,
                    rho_coarse_com_mean = rho_coarse_com_mean,
                    a = a_inside_new,
                    G = G,
                    use_TSC = use_TSC,
                    n_threads = n_threads
                )
                future_cooling = executor.submit(
                    cooling_heating_step_baryons_py, #grad_phi5.cooling_heating_step
                    pos_baryon=x_inside_new[baryon_mask_inside] - box_min_fine,
                    vel_baryon=u_inside_half[baryon_mask_inside],
                    masses_baryon=masses_inside[baryon_mask_inside],
                    rho_b_coarse_com_mean=rho_b_coarse_com_mean,
                    U_baryon=U_b_inside,
                    Lbox_Mpc=L_fine,
                    Ng=Nf,
                    dt_Myr=dt_inside,
                    a=a_inside_new,
                    use_TSC=use_TSC,
                    n_threads=n_threads,
                    smooth=f_smooth # smoothing parameter for fine grid
                )

                acc_in, max_tot_od_fine = future_phi.result()
                cooling_results = future_cooling.result()

            g_inside_new = acc_in / a_inside_new**2
            U_b_inside, T_b_inside, gP_b_inside_new, P_b_inside, diagnosis3, _, dt_target_inside_new, max_b_od_fine, _, temp_by_b_od_fine, S_max_fine = cooling_results
            u_inside_new = u_inside_half + 0.5 * dt_inside * (g_inside_new - 2 * H_inside_half * u_inside_half)
            u_inside_new[baryon_mask_inside] += 0.5 * dt_inside * gP_b_inside_new
            x_inside = x_inside_new
            u_inside = u_inside_new
            g_inside = g_inside_new
            a_inside = a_inside_new
            gP_b_inside = gP_b_inside_new
            Zsol_max_fine = max(Zsol_max_fine, diagnosis3.get('Zsol_max',0))
            nH_max_fine = max(nH_max_fine, diagnosis3.get('nH_max',0))

        # END LOOP

        dt_dyn_fine = f_dyn / np.sqrt(max(4.0*np.pi*G*max_tot_od_fine*total_mass/(a*L)**3, 1e-40))
        dt_target_fine = min(dt_dyn_fine, dt_target_inside_new)
        diagnosis2['max_tot_od_fine'] = max_tot_od_fine
        diagnosis2['dt_dyn_fine'] = dt_dyn_fine
        diagnosis1['max_b_od_fine'] = max_b_od_fine
        diagnosis1['Zsol_max_fine'] = Zsol_max_fine
        diagnosis1['nH_max_fine'] = nH_max_fine
        diagnosis1['dt_cfl_fine'] = dt_target_inside_new
        diagnosis1['dt_target_fine'] = dt_target_fine
        diagnosis1['infall'] = mask_inside.sum()
        diagnosis1['infall_b'] = mask_b_inside.sum()

        inside_mask_interior = mask_interior[mask_inside]
        inside_b_mask_interior_b = mask_b_interior[mask_b_inside]
        u_new[mask_inside & mask_interior] = u_inside_new[inside_mask_interior]
        g_new[mask_inside & mask_interior] = g_inside_new[inside_mask_interior]
        x_new[mask_inside & mask_interior] = x_inside_new[inside_mask_interior]
        U_b_new[mask_b_inside & mask_b_interior] = U_b_inside[inside_b_mask_interior_b] # This allocates only those in mas_interior
        T_b_new[mask_b_inside & mask_b_interior] = T_b_inside[inside_b_mask_interior_b]
        P_b_new[mask_b_inside & mask_b_interior] = P_b_inside[inside_b_mask_interior_b]
        gP_b_new[mask_b_inside & mask_b_interior] = gP_b_inside[inside_b_mask_interior_b]

    return x_new, u_new, g_new, a_new, PE, U_b_new, T_b_new, gP_b_new, P_b_new, diagnosis1 | diagnosis2, epochs, dt_target_new, max_b_od, max_tot_od, temp_by_b_od, S_max

In [ ]:
# @title
# Simulation loop
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.animation as animation
from IPython.display import HTML
import io, sys, subprocess, cProfile
import time as tm
from datetime import datetime

# Define a Tee class to write to multiple streams
class Tee:
    def __init__(self, *streams):
        self.streams = streams
    def write(self, text):
        for s in self.streams:
            s.write(text)
            s.flush()
    def flush(self):
        for s in self.streams:
            s.flush()


@njit
def total_angular_momentum(x, u, masses, a): # x, u: shape (N, 3), masses: shape (N,)
    return np.sum(np.cross(x, u) * masses[:, np.newaxis], axis=0) * a**2
@njit
def measure_flattening(positions):
    a = np.std(positions[:,0])
    b = np.std(positions[:,1])
    c = np.std(positions[:,2])
    return a,b,c

# Run simulation
positions_hist = []
scale_hist = []
time_hist = []
temp_hist = []
pressure_hist = []
KE_hist = []
PE_hist = []
TE_hist = []
virial_hist = []
flat_a_hist = []
flat_b_hist = []
flat_c_hist = []

body_velocities_cos = body_velocities / Mpc_km * Myr_s # turn all velocities into pc/yr
center = np.mean(body_positions, axis=0) # center of mass of system
body_positions %= L # wrap all particles into the box
res = max(steps // 100, 1) # resolution for animation (not for simulation!)
scale = a_init
time = 0 # Myr
dt_target = epsilon / a_init # Myr
H_a = H0_cos * E(scale) #1/Myr
vel_max = 0.0 # for identification of max speed

eng = grad_phi5.PMEngine(Ngrid=Ng, box_size=L, use_TSC=use_TSC, n_threads=n_threads)
acc, _, PE, _ = eng.step(pos=body_positions, masses=masses, total_mass=masses.sum(), a=scale, G=G)
acc -= 2 * H_a * body_velocities_cos # Slowing required in comoving coordinates
L_ang = total_angular_momentum(body_positions, body_velocities_cos, masses, scale)
U_b = init_internal_energy(body_positions, baryon_set, masses=masses, Lbox_Mpc=L, Ng=Ng, a_init=a_init, model="pre_reion_adiabatic", T_floor=50.0, n_threads=n_threads)  # neutral gas: mu≈1.22
grad_phi5.set_cosmology_params(H0_cos=H0_cos, Omega_m=Ω_m, Omega_r=Ω_r, Omega_k=Ω_k, Omega_lambda=Ω_Λ, Omega_b=Ω_b)
grad_phi5.set_n_jeans(n = n_jeans)
grad_phi5.set_subgrid_config(
    cube_fine = cube_fine,
    center_fine = center_fine,
    refinement = refinement,
    od_threshold = od_threshold,
    subcycles = subcycles,
    f_smooth = f_smooth,
    margin = margin
)

rho_fine_final = None
acc_P = np.zeros_like(body_velocities_cos[baryon_set])

sim_log = io.StringIO()
tee = Tee(sys.stdout, sim_log)
print(datetime.now(), file=tee)
print(f"N_bodies: {N_bodies:,}, grid cells: {Ng**3:,}, max particle mass: {masses.max(): .3e} m_sun", file=tee)
print_subgrid_parameters(cube_size=cube_fine, r=refinement, Lc=L, Nc=Ng, margin=margin, file=tee)
print(f"Initial scale factor a: {a_init:.4f}", file=tee)
print(f"Total system mass: {M_total:.3e} m_sun", file=tee)
print(f"Initial angular momentum: {np.round(L_ang)} Mpc²⋅m_sun/Myr", file=tee)

def simulation_loop(): # Simulation
    global scale, U_b, body_positions, body_velocities_cos, acc, acc_P, vel_max, time, dt_target, T, P, subcycles, rho_fine_final

    animation_dt = t_universe / 100
    animation_t = 0.0
    step = 0
    combined_step_fct = combined_step_parallel_py if use_Python else grad_phi5.combined_step
    while scale < a_final:
        H_a = H0_cos * E(scale) #1/Myr
        dt = np.clip(dt_target, epsilon_fine / H_a, epsilon / H_a)  # Myr
        time += dt

        (body_positions, body_velocities_cos, acc, scale, PE,
        U_b, T, acc_P, P, diag, epochs, dt_target_new,
        max_b_od, max_tot_od, temp_by_b_od, S_max) = combined_step_fct(
            x=body_positions,
            u=body_velocities_cos,
            g=acc,
            gP_b=acc_P,
            U_b=U_b,
            baryon_mask=baryon_set,
            masses=masses,
            total_mass=masses.sum(),
            L=L,
            Ng=Ng,
            dt=dt,
            a=scale,
            G=G,
            eng=eng,
            use_TSC=use_TSC,
            n_threads=n_threads
        )

        # Unpack diagnosis
        max_tot_od = diag.get('max_tot_od', max_tot_od)
        max_tot_od_fine = diag.get('max_tot_od_fine', None)
        max_b_od_fine = diag.get('max_b_od_fine', None)
        cooling_fine_ms = diag.get('cooling_fine_ms', np.nan)
        gravity_fine_ms = diag.get('gravity_fine_ms', np.nan)
        overlap_fine_ms = diag.get('overlap_fine_ms', np.nan)
        cooling_ms = diag.get('cooling_ms', np.nan)
        gravity_ms = diag.get('gravity_ms', np.nan)
        overlap_ms = diag.get('overlap_ms', np.nan)

        if step % 100 == 0 or scale >= a_final:
            print(file=tee)
            formatted_list = '[' + ', '.join(f"{t:,.0f}" for t in temp_by_b_od) + ']'
            print(f"z={1/scale-1:.1f}, step={step:,}, dt/dt_target={dt/dt_target:.3f}, T min/max={np.min(T):,.0f}/{np.max(T):,.0f} K, S_max={S_max:.3f}, max b/tot od={max_b_od:.1f}/{max_tot_od:.1f}", end='', file=tee)
            if max_tot_od_fine is not None and max_b_od_fine is not None:
                print(f", MR: max b/tot od={max_b_od_fine:.1f}/{max_tot_od_fine:.1f}, dt/dt_dyn/cfl={dt/subcycles/diag.get('dt_dyn_fine', np.nan):.3f}/{dt/subcycles/diag.get('dt_cfl_fine', np.nan):.3f}, max nH/Zsol={diag.get('nH_max_fine',np.nan):.3f}/{diag.get('Zsol_max_fine', np.nan):.3f}, infall b/tot={diag.get('infall_b',np.nan):,}/{diag.get('infall',np.nan):,}", end='', file=tee)
            else:
                print(f", no MR", end='', file=tee)
            #print(f", T_jeans min/mwm/mean/max={diag.get('T_jeans_min', np.nan):.0f}/{diag.get('T_jeans_mwm', np.nan):.0f}/{diag.get('T_jeans_mean', np.nan):.0f}/{diag.get('T_jeans_max', np.nan):.0f}", end='', file=tee)
            #print(f", f_sh min/mwm/mean/max={diag.get('f_sh_min', np.nan):.3f}/{diag.get('f_sh_mwm', np.nan):.3f}/{diag.get('f_sh_mean', np.nan):.3f}/{diag.get('f_sh_max', np.nan):.3f}", end='', file=tee)
            print(f", Zsol min/mean/max={diag['Zsol_min']:.3f}/{diag['Zsol_mean']:.3f}/{diag['Zsol_max']:.3f}, temp by od={formatted_list}", end='', file=tee)
            #print(file=tee)
            #print(f"coarse: gravity/thermo/overlap={gravity_ms/1000:.1f}/{cooling_ms/1000:.1f}/{overlap_ms/1000:.1f} s, fine: gravity/thermo/overlap={gravity_fine_ms/1000:.1f}/{cooling_fine_ms/1000:.1f}/{overlap_fine_ms/1000:.1f} s", end='', file=tee)
        if time >= animation_t or scale >= a_final:
            animation_t += animation_dt
            vel_max = np.fmax(vel_max, grad_phi5.velocity_stat(body_velocities_cos, n_threads) * scale)
            KE = 0.5 * scale**2 * np.sum(masses * np.sum(body_velocities_cos**2, axis=1))  # M☉⋅Mpc²/Myr²
            KE_hist.append(KE.astype(np.float32))
            PE_hist.append(PE)
            TE_hist.append(np.sum(masses[baryon_set] * U_b) * Myr_to_s**2 / Mpc_to_cm**2) # m_sun⋅Mpc²/Myr²
            virial_hist.append(2*KE.astype(np.float32)/np.abs(PE))
            positions_hist.append((body_positions[grid_downsampled]-center).astype(np.float32))
            scale_hist.append(scale)
            time_hist.append(time)
            aa, bb, cc = measure_flattening(body_positions.astype(np.float32))
            flat_a_hist.append(aa * scale)
            flat_b_hist.append(bb * scale)
            flat_c_hist.append(cc * scale)
            tmp = np.zeros_like(body_positions[:,0])
            tmp[baryon_set] = T
            temp_hist.append(tmp[grid_downsampled].astype(np.float32))
            tmp = np.zeros_like(body_positions[:,0])
            tmp[baryon_set] = P
            pressure_hist.append(tmp[grid_downsampled].astype(np.float32))
        if step % res == 0:
            print('.', end='', flush=True, file=tee)
        step += 1
        dt_target = dt_target_new

    rho_fine_final = create_rho_fine(cube_center=center_fine, cube_size=cube_fine, r=refinement, Lc=L, Nc=Ng, pos_all=body_positions, masses_all=masses, use_TSC=use_TSC)

    print(file=tee)
    return

start_time = tm.time()
simulation_loop()
end_time = tm.time()
elapsed = end_time - start_time

L_ang = total_angular_momentum(body_positions, body_velocities_cos, masses, scale)
print(file=tee)
print(f"Final angular momentum: {np.round(L_ang)} Mpc²⋅m_sun/Myr", file=tee)
print(f"Final scale factor a: {scale:.4f}", file=tee)
print(f"Final scale factor for animation: {scale_hist[-1]:.4f}", file=tee)
print(f"Highest v_pec: {vel_max/c_cos:.6f} c", file=tee)
print(f"Simulation took {elapsed:.1f} seconds.", file=tee)

__Visualize Energy__

In [ ]:
# @title
import base64
from io import BytesIO

def create_html(fig):
    # Save the figure to a buffer
    buf = BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)

    # Encode the image as base64
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()

    # Create HTML content
    return f"""
    <!DOCTYPE html>
    <html>
    <head><title>Matplotlib Figure</title></head>
    <body>
    <img src="data:image/png;base64,{img_base64}" alt="Matplotlib Figure">
    </body>
    </html>
    """

In [ ]:
# @title
fig, axes = plt.subplots(1, 2, figsize=(10, 4))  # 1 row, 2 columns
# === LEFT PANEL: Energy + virial with twin y-axis ===
ax1 = axes[0]
ax1.plot(time_hist, KE_hist, label='Kinetic Energy', color='tab:blue')
ax1.plot(time_hist, PE_hist, label='Potential Energy', color='tab:orange')
ax1.plot(time_hist, TE_hist, label='Thermal Energy')
ax1.plot(time_hist, np.array(KE_hist) + np.array(PE_hist) + np.array(TE_hist), label='Total Energy', color='tab:green')
ax1.set_xlabel("Time [Myr]")
ax1.set_ylabel("Energy [M$_\\odot$⋅Mpc²⋅Myr⁻²]")

# Twin axis for virial ratio
ax2 = ax1.twinx()
ax2.plot(time_hist, virial_hist, label='Virial ratio', color='tab:red', linestyle='--')
ax2.set_ylabel("Virial ratio")

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right")

ax1.set_title("Energy Evolution & Virial Balance")

# === RIGHT PANEL: Flattening ===
ax_flat = axes[1]
ax_flat.plot(time_hist, flat_a_hist, label='$\\sigma_x$')
ax_flat.plot(time_hist, flat_b_hist, label='$\\sigma_y$')
ax_flat.plot(time_hist, flat_c_hist, label='$\\sigma_z$')
ax_flat.set_xlabel("Time [Myr]")
ax_flat.set_ylabel("Stdev per axis [Mpc]")
ax_flat.set_title("System Expansion Over Time")
ax_flat.legend()

plt.tight_layout()
plt.show()
energy_html = create_html(fig)

__End of simulation delta field__

In [ ]:
# @title
import plotly.io as pio
delta_end = grad_phi5.deposit_delta(positions=body_positions, masses=masses, Ngrid=Ng, L=L, use_TSC=use_TSC, n_threads=n_threads)
slice_index = (Ng // 2)
delta_2D_end = delta_end[:, :, slice_index]
body_z = body_positions[:, 2]
body_mask = (body_z > slice_index/Ng*L - qg*dx_init/2) & (body_z < slice_index/Ng*L + qg*dx_init/2)

print("δ Field today, from n-body simulation:")
print(f"np.mean(delta) = {np.mean(delta_end):.4f}")
print(f"np.max(delta) = {np.max(delta_end):.4f}")
print(f"np.min(delta) = {np.min(delta_end):.4f}")
print(f"np.std(delta) = {np.std(delta_end):.4f}")
print(f"delta.shape = {delta_end.shape}")
print(f"np.std(delta_2D) = {np.std(delta_2D_end):.4f}")

fig = plt.figure(figsize=(10, 4)) # Plot 2D slice of the field
plt.subplot(1, 2, 1)

max_log = np.percentile(np.log(delta_2D_end.T+1+1e-10), 99.5)
helper = np.clip(np.log(delta_2D_end.T+1+1e-10), -max_log, +max_log)
im = plt.imshow(helper, origin='lower', extent=[0, L, 0, L], cmap='Blues')
plt.colorbar(im, label='ln(δ) (today)', fraction = 0.026)
plt.title(f"δ Field today (slice {slice_index})") # Labels and title
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")

plt.subplot(1, 2, 2)
plt.scatter(body_positions[body_mask, 0], body_positions[body_mask, 1], s=1, alpha=0.5)
plt.title(f"Particle Distribution (Slice {slice_index})")
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.xlim(0, L)
plt.ylim(0, L)
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.show()
deltafield_html = create_html(fig)

In [ ]:
# @title
# rho_fine_com_interior chart
from scipy.ndimage import uniform_filter
if rho_fine_final is not None:
    smoothing = 1 # 1 leaves the field unchanged
    rho_com_mean = masses.sum()/L**3
    smooth_rho = uniform_filter(rho_fine_final, size=smoothing, mode='reflect')
    smooth_rho[smooth_rho < 0] = 0
    print(f"Fine grid overdensity today, smoothed over {smoothing} cells:")
    print(f"Mean  = {np.mean(smooth_rho)/rho_com_mean:.1f}")
    print(f"Max   = {np.max(smooth_rho)/rho_com_mean:.3e}")
    print(f"Min   = {np.min(smooth_rho)/rho_com_mean:.3e}")
    print(f"Shape = {smooth_rho.shape}")

    slice_index = int(smooth_rho.shape[0]*0.5)
    od_2D_end = smooth_rho[:, :, slice_index]/rho_com_mean
    fig = plt.figure(figsize=(5,4)) # Plot 2D slice of the field
    max_log = np.percentile(od_2D_end.T, 99.5)
    helper = np.clip(od_2D_end.T, 0, +max_log)
    im = plt.imshow(helper, origin='lower', extent=[0, cube_fine, 0, cube_fine], cmap='plasma')
    plt.colorbar(im, label='fine grid overdensity', fraction = 0.026)
    plt.title(f"Fine grid overdensity today (slice {slice_index})") # Labels and title
    plt.xlabel("x [Mpc]")
    plt.ylabel("y [Mpc]")
    fine_grid_od_html = create_html(fig)

__Baryon Temperature Distribution__

In [ ]:
# @title
# Baryon Temperature Distribution
import matplotlib.colors as colors

baryon_pos = body_positions[baryon_set]
baryon_masses = masses[baryon_set]
body_z = baryon_pos[:, 2]
body_mask = (body_z > slice_index/Ng*L - qg*dx_init/2) & (body_z < slice_index/Ng*L + qg*dx_init/2)
rho_b_com_grid, _dx = grad_phi5.deposit_density(pos_Mpc=baryon_pos, masses=baryon_masses, Lbox_Mpc=L, Ng=Ng, use_TSC=use_TSC)
rho_b_com_grid *= Msun_to_g / Mpc_to_cm**3
baryon_rho = grad_phi5.gather_scalar(grid=rho_b_com_grid, pos_Mpc=baryon_pos, Lbox_Mpc=L, dx=_dx, use_TSC=use_TSC, n_threads=n_threads)
print(f"Mean baryon density: {np.mean(rho_b_com_grid):.3e} g/cm3")
print(f"Mean baryon temp: {np.mean(T):.3e} K")

fig = plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)

sc = plt.scatter(
    baryon_pos[body_mask, 0],
    baryon_pos[body_mask, 1],
    c=T[body_mask],
    cmap='plasma',    # Or 'viridis', 'inferno', etc.
    s=1,
    alpha=0.5
)
plt.title(f"Baryon Temperature Distribution (Slice {slice_index})")
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.xlim(0, L)
plt.ylim(0, L)
plt.gca().set_aspect('equal')
cbar = plt.colorbar(sc)
cbar.set_label("Temperature [K]")

plt.subplot(1, 2, 2)
sc = plt.scatter(
    baryon_rho/np.mean(rho_b_com_grid),
    T,
    c=P,
    cmap='plasma',
    s=1,
    alpha=0.2,
    norm=colors.LogNorm(vmin=P.min(), vmax=P.max())
)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Overdensity")
plt.ylabel("Temp (K)")
plt.title("Density / Temperature correlation")
plt.grid(True, which="both", ls="--", alpha=0.5)
cbar = plt.colorbar(sc) # Add colorbar
cbar.set_label("log10(Pressure) [erg/cm³]")
plt.tight_layout()
plt.show()
thermo_html = create_html(fig)

__Animation__

In [ ]:
# @title
import plotly.graph_objects as go

R_scale = L/2
phys_scale = True
menues = [{
    "type": "buttons",
    "buttons": [
        {
            "label": "Play",
            "method": "animate",
            "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}]
        },
        {
            "label": "XY Plane",
            "method":"relayout",
            "args": [{"scene.camera.eye": {"x": 0, "y": 0, "z": 2},
                    "scene.camera.up": {"x": 0, "y": 1, "z": 0},
                    "scene.camera.projection.type": "orthographic",
                    "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
            }]
        },
        {
            "label": "XZ Plane",
            "method": "relayout",
            "args": [{"scene.camera.eye": {"x": 0, "y": -2, "z": 0},
                    "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                    "scene.camera.projection.type": "orthographic",
                    "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
            }]
        },
        {
            "label": "YZ Plane",
            "method": "relayout",
            "args": [{"scene.camera.eye": {"x": 2, "y": 0, "z": 0},
                    "scene.camera.projection.type": "orthographic",
                    "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
            }]
        },
        {
            "label": "3D View",
            "method": "relayout",
            "args": [{"scene.camera.eye": {"x": 1.4, "y": -1.8, "z": 1.3},
                    "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                    "scene.camera.projection.type": "perspective",
                    "scene.aspectratio": {"x": 1.2, "y": 1.2, "z": 1.2}
            }]
        }
    ],
}]

def create_sphere_mesh(center, radius, color='gray', opacity=0.1, resolution=40):
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v)).flatten()
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v)).flatten()
    z = center[2] + radius * np.outer(np.ones_like(u), np.cos(v)).flatten()
    i, j = np.meshgrid(np.arange(resolution - 1), np.arange(resolution - 1))
    i = i.flatten()
    j = j.flatten()
    faces = []
    for k in range(len(i)):
        a = i[k] * resolution + j[k]
        b = a + 1
        c = a + resolution
        d = c + 1
        faces.append([a, b, d])
        faces.append([a, d, c])
    i, j, k = np.array(faces).T
    return go.Mesh3d(
        x=x.astype(np.float16), y=y.astype(np.float16), z=z.astype(np.float16),
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        name='Expansion Sphere',
        showscale=False
    )

# Create Plotly animation frames
frames = []
for i, pos in enumerate(positions_hist):
    aa = scale_hist[i] if phys_scale else 1.0
    pos = pos.astype(np.float16)
    ref_sphere = create_sphere_mesh(np.zeros(3), R_scale * scale_hist[i], color='gray')
    frames.append(go.Frame(
        data=[
            go.Scatter3d(x=pos[~baryon_set[grid_downsampled], 0]*aa, y=pos[~baryon_set[grid_downsampled], 1]*aa, z=pos[~baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
            go.Scatter3d(x=pos[baryon_set[grid_downsampled], 0]*aa, y=pos[baryon_set[grid_downsampled], 1]*aa, z=pos[baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=1), name='Baryon'),
            ref_sphere
        ],
        name=f'frame{i}',
        layout=go.Layout(title_text='N-body Simulation')
    ))

# Initial frame
init_pos = positions_hist[0].astype(np.float16)
aa = scale_hist[0] if phys_scale else 1.0
ref_sphere = create_sphere_mesh(np.zeros(3), R_scale * scale_hist[0], color='gray')
fig = go.Figure(
    data=[
        go.Scatter3d(x=init_pos[~baryon_set[grid_downsampled], 0]*aa, y=init_pos[~baryon_set[grid_downsampled], 1]*aa, z=init_pos[~baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
        go.Scatter3d(x=init_pos[baryon_set[grid_downsampled], 0]*aa, y=init_pos[baryon_set[grid_downsampled], 1]*aa, z=init_pos[baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=1), name='Baryon'),
        ref_sphere
    ],
    layout=go.Layout(
        title='N-body Simulation',
        width = 1000,
        height = 800,
        scene=dict(
            xaxis=dict(title='x [Mpc]', range=[-R_scale, R_scale]),
            yaxis=dict(title='y [Mpc]', range=[-R_scale, R_scale]),
            zaxis=dict(title='z [Mpc]', range=[-R_scale, R_scale]),
            camera=dict(
                eye=dict(x=1.4, y=-1.8, z=1.3),
                up=dict(x=0, y=0, z=-1),
                projection=dict(type='perspective')
            ),
            aspectratio=dict(x=1.2, y=1.2, z=1.2)
        ),
        updatemenus=menues,
        sliders=[{
                    "steps": [
                        {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                         "label": f"{time_hist[i]:,.0f} Myr",
                         "method": "animate"
                         } for i in range(0,len(positions_hist),1)
                    ],
                }],
    ),
    frames=frames
)
fig.show()
#fig.write_html('DFS-v9-comoving.html') # Save to HTML
plot_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

__Baryon temperature animation__

In [ ]:
# @title
R_scale = L/2
phys_scale = False
body_pos = positions_hist[-1].astype(np.float16)
baryon_pos = body_pos[baryon_set[grid_downsampled]]
body_temp = temp_hist[-1]
baryon_temp = body_temp[baryon_set[grid_downsampled]]
#log_temp = np.log10(np.clip(baryon_temp, 1, None))  # avoid log(0)
cmin = 0
cmax = np.percentile(baryon_temp, 95)

# Create Plotly animation frames
frames = []
for i, pos in enumerate(positions_hist):
    aa = scale_hist[i] if phys_scale else 1.0
    body_pos = pos.astype(np.float16)
    baryon_pos = body_pos[baryon_set[grid_downsampled]]
    body_temp = temp_hist[i]
    baryon_temp = body_temp[baryon_set[grid_downsampled]]
    #log_temp = np.log10(np.clip(baryon_temp, 1, None))  # avoid log(0)
    frames.append(go.Frame(
            data=[
            go.Scatter3d(
                x=baryon_pos[:, 0] * aa,
                y=baryon_pos[:, 1] * aa,
                z=baryon_pos[:, 2] * aa,
                mode='markers',
                marker=dict(
                    size=1,
                    color=baryon_temp,
                    colorscale='plasma',
                    cmin=cmin,
                    cmax=cmax,
                    colorbar=dict(
                        title="Temp [K]",
                        #tickvals=[2, 4, 6, 8],     # example ticks
                        #ticktext=["10²", "10⁴", "10⁶", "10⁸"]
                    ),
                    opacity=0.8
                ),
                name='Baryon'
            )
        ],
        name=f'frame{i}',
        layout=go.Layout(title_text='N-body Simulation - Baryon Temperature')
    ))

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=baryon_pos[:, 0] * aa,
            y=baryon_pos[:, 1] * aa,
            z=baryon_pos[:, 2] * aa,
            mode='markers',
            marker=dict(
                size=1,
                color=baryon_temp,
                colorscale='plasma', # 'Viridis'
                cmin=cmin,
                cmax=cmax,
                colorbar=dict(
                    title="Temp [K]",
                    #tickvals=[2, 4, 6, 8],     # example ticks
                    #ticktext=["10²", "10⁴", "10⁶", "10⁸"]
                ),
                opacity=0.8
            ),
            name='Baryon'
        ),
    ],
    layout=go.Layout(
        title='N-body Simulation - Baryon Temperature',
        width=1000,
        height=800,
        scene=dict(
            xaxis=dict(title='x [Mpc]', range=[-R_scale, R_scale]),
            yaxis=dict(title='y [Mpc]', range=[-R_scale, R_scale]),
            zaxis=dict(title='z [Mpc]', range=[-R_scale, R_scale]),
            camera=dict(
                eye=dict(x=1.4, y=-1.8, z=1.3),
                up=dict(x=0, y=0, z=-1),
                projection=dict(type='perspective')
            ),
            aspectratio=dict(x=1.2, y=1.2, z=1.2)
        ),
        updatemenus=menues,
        sliders=[{
            "steps": [
                {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": f"{time_hist[i]:,.0f} Myr",
                    "method": "animate"
                    } for i in range(0,len(positions_hist),1)
            ],
        }],

    ),
    frames=frames
)
fig.show()
thermo_animation_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

__Halo identification__

In [ ]:
# @title
from scipy.spatial import cKDTree

def fof_groups(positions, linking_length):
    tree = cKDTree(positions)
    N = len(positions)
    groups = []
    visited = np.zeros(N, dtype=bool)

    for i in range(N):
        if visited[i]:
            continue
        group = [i]
        queue = [i]
        visited[i] = True
        while queue:
            j = queue.pop()
            neighbors = tree.query_ball_point(positions[j], linking_length)
            for n in neighbors:
                if not visited[n]:
                    visited[n] = True
                    group.append(n)
                    queue.append(n)
        groups.append(np.array(group))
    result = [g for g in groups if len(g) >= 20]
    return sorted(result, key=len, reverse=True)

In [ ]:
# @title
# Halo analysis function
def analyze_halo(pos_halo, vel_halo, masses_halo, a, G_cos):
    """
    Analyze halo properties for a group of particles.

    Parameters:
        positions : array of all particle positions [Mpc, physical]
        velocities: array of all particle velocities [Mpc/Myr]
        mass      : mass of a single particle [Msun]
        a         : scale factor

    Returns:
        radius         : max distance from center [Mpc]
        L              : angular momentum vector [Msun·Mpc²/Myr]
        omega          : angular velocity vector [rad/Myr]
        overdensity    : (rho_halo / rho_crit(a)) - 1
        lambda_bullock : dimensionless spin parameter
    """
    N = len(pos_halo)
    halo_mass = masses_halo.sum()

    if N == 0: return np.nan, np.full(3, np.nan), np.nan, np.nan, np.nan, np.nan

    # Center of mass
    x_cm = np.average(pos_halo, axis=0, weights=masses_halo)
    v_cm = np.average(vel_halo, axis=0, weights=masses_halo)

    # Relative positions and velocities
    x_rel = (pos_halo - x_cm) * a
    v_rel = (vel_halo - v_cm) * a

    # Radius = max distance from center
    radius = np.max(np.linalg.norm(x_rel, axis=1))  # Mpc

    # Angular momentum
    L_ang = np.sum(np.cross(x_rel, v_rel)*masses_halo[:, np.newaxis], axis=0)  # Msun·Mpc²/Myr

    # Mean density in halo
    V = (4/3) * np.pi * radius**3
    rho_halo = halo_mass / V  # Msun / Mpc³

    # Critical density at scale factor a
    rho_mean_matter = Ω_m * rho_crit0 * E(a)**2 # M_sun / Mpc³
    overdensity = rho_halo / rho_mean_matter - 1

    # Bullock spin parameter λ'
    V_circ = np.sqrt(G_cos * halo_mass / radius)  # Mpc/Myr
    lambda_bullock = np.linalg.norm(L_ang) / (np.sqrt(2) * halo_mass * V_circ * radius) # dimensionless

    # Computes the shape (inertia) tensor of a halo.
    xx = x_rel[:, 0]
    yy = x_rel[:, 1]
    zz = x_rel[:, 2]

    I_tensor = np.zeros((3, 3))
    I_tensor[0, 0] = np.sum(xx * xx * masses_halo)
    I_tensor[0, 1] = np.sum(xx * yy * masses_halo)
    I_tensor[0, 2] = np.sum(xx * zz * masses_halo)
    I_tensor[1, 0] = I_tensor[0, 1]
    I_tensor[1, 1] = np.sum(yy * yy * masses_halo)
    I_tensor[1, 2] = np.sum(yy * zz * masses_halo)
    I_tensor[2, 0] = I_tensor[0, 2]
    I_tensor[2, 1] = I_tensor[1, 2]
    I_tensor[2, 2] = np.sum(zz * zz * masses_halo)

    #omega_vec = np.linalg.inv(I_tensor) @ L_ang  # rad/Myr
    #omega = np.linalg.norm(omega_vec)

    try:
        omega_vec = np.linalg.inv(I_tensor) @ L_ang
        omega = np.linalg.norm(omega_vec)
    except np.linalg.LinAlgError:
        omega_vec = np.full_like(L_ang, np.nan)
        omega = np.nan

    eigvals, eigvecs = np.linalg.eigh(I_tensor / halo_mass) # Normalize to halo mass
    # Sort eigenvalues: a^2 ≥ b^2 ≥ c^2
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    a, b, c = np.sqrt(eigvals)
    triax = (a**2-b**2)/(a**2-c**2)
    return radius, L_ang, omega, overdensity, lambda_bullock, triax


In [ ]:
# @title
import io

# Halo statistics
# Expected values (Tinker)   >1e0    >1e1   >1e2   >1e3   >1e4   >1e5   >1e6    >1e7       >1e8       >1e9       >1e10      >1e11      >1e12      >1e13      >1e14
halos_per_Mpc3 = np.array([np.inf, np.inf,np.inf,np.inf,np.inf,np.inf,np.inf, 6.438e+01, 7.658e+00, 9.321e-01, 1.172e-01, 1.530e-02, 2.054e-03, 2.586e-04, 1.972e-05])

buffer = io.StringIO()
final_pos = body_positions.astype(np.float32)
final_vel = body_velocities_cos.astype(np.float32)
mean_L = scale * L/Np
linking_length = 0.12 * mean_L  # Adjust based on mean interparticle separation
#groups = fof_groups(final_pos, linking_length)
groups = grad_phi5.fof_groups(final_pos, linking_length)
n_weights = len(halos_per_Mpc3)
weight_cnt = np.zeros(n_weights)

print(f"N_bodies: {N_bodies:,}, grid cells: {Ng**3:,}, L: {L} Mpc, mean particle sep: {mean_L:.3f} Mpc, linking length: {linking_length:.3f} Mpc", file=buffer)
print(f"a_init: {a_init:.3f}, a_final: {scale:.3f}, steps: {steps:,}", file=buffer)
print(f"Max particle mass: {masses.max(): .3e} m_sun, Total system mass: {M_total:.3e} m_sun", file=buffer)
print(f"Highest v_pec: {vel_max/c_cos:.6f} c", file=buffer)
print(f"δ field today (mean/max/min/std): {np.mean(delta_end):.3f} / {np.max(delta_end):.1f} / {np.min(delta_end):.3f} / {np.std(delta_end):.3f}", file=buffer)
print(f"Random seed for simulation: {seed}", file=buffer)
print(file=buffer)

for i, g in enumerate(groups):
    halo_mass = masses[g].sum()
    for j in range(n_weights-1,0,-1):
        if halo_mass > 10**j:
            weight_cnt[j] += 1
            break

print(f"Found {len(groups)} halos", file=buffer)
print(f"Halo statistics for box volume: {L**3} Mpc³", file=buffer)
min_mass = masses.min()
for i in range(n_weights-1,0,-1): # iterate over weight classes
    if 10**i < min_mass: # if weight class < particle mass: break
        break
    print(f"> {10**i:.0e}: {weight_cnt[i].astype(int)} halos, expected {halos_per_Mpc3[i]*L**3:.1f}", file=buffer)
print(file=buffer)
final_temp = np.zeros_like(body_positions[:,0])
final_pressure = np.zeros_like(body_positions[:,0])
final_temp[baryon_set] = T
final_pressure[baryon_set] = P

for i, g in enumerate(groups):
    if i>20:
        break
    halo_pos = final_pos[g]
    halo_vel = final_vel[g]
    halo_masses = masses[g]
    halo_center = np.average(halo_pos, axis=0, weights=halo_masses)
    halo_mass = masses[g].sum()
    halo_baryons = np.intersect1d(g, baryon_indices)
    baryon_count = len(halo_baryons)
    av_baryon_temp = np.mean(final_temp[halo_baryons])
    radius, L_ang, omega, overdensity, spin, triax = analyze_halo(halo_pos, halo_vel, halo_masses, scale_hist[-1], G) # CDM+baryons
    radius_b, L_ang_b, omega_b, overdensity_b, spin_b, triax_b = analyze_halo(final_pos[halo_baryons], final_vel[halo_baryons], masses[halo_baryons], scale_hist[-1], G) # baryons only
    overdensity_b *= Ω_m / Ω_b
    print(f"#{i}: Halo with {len(g):,} particles, mass = {halo_mass:.3e} Msun, position = {halo_center[0]:.3f}/{halo_center[1]:.3f}/{halo_center[2]:.3f}, %baryons = {baryon_count/len(g):.4f}", file=buffer)
    print(f"              CDM+baryons/baryons only", file=buffer)
    print(f"Radius:       {radius:.2f}/{radius_b:.2f} Mpc", file=buffer)
    print(f"Temp:         {av_baryon_temp:,.0f} K", file=buffer)
    print(f"|L|:          {np.linalg.norm(L_ang):.2e}/{np.linalg.norm(L_ang_b):.2e} Msun·Mpc²/Myr", file=buffer)
    print(f"|omega|:      {omega:.3e}/{omega_b:.3e} rad/Myr, {2*np.pi/1000/omega:.1f}/{2*np.pi/1000/omega_b:.1f} Gy per revolution", file=buffer)
    print(f"Triaxiality:  {triax:.3f}/{triax_b:.3f} (0=oblate, 1=prolate)", file=buffer)
    print(f"Overdensity:  {overdensity:.2f}/{overdensity_b:.2f}", file=buffer)
    print(f"Spin λ':      {spin:.4e}/{spin_b:.4e}", file=buffer)
    print(file=buffer)

report_text = buffer.getvalue()
print(report_text)

__Disk-shaped halos__

In [ ]:
# @title
for i, g in enumerate(groups):
    if i>100:
        break
    halo_pos = final_pos[g]
    halo_vel = final_vel[g]
    halo_masses = masses[g]
    halo_center = np.mean(final_pos[g], axis=0)
    halo_mass = masses[g].sum()
    halo_baryons = np.intersect1d(g, baryon_indices)
    baryon_count = len(halo_baryons)
    if baryon_count < 10: continue
    av_baryon_temp = np.mean(final_temp[halo_baryons])
    radius, L_ang, omega, overdensity, spin, triax = analyze_halo(halo_pos, halo_vel, halo_masses, scale_hist[-1], G) # CDM+baryons
    radius_b, L_ang_b, omega_b, overdensity_b, spin_b, triax_b = analyze_halo(final_pos[halo_baryons], final_vel[halo_baryons], masses[halo_baryons], scale_hist[-1], G) # baryons only
    overdensity_b *= Ω_m / Ω_b
    if triax_b < 0.3 and radius_b != np.nan:
        print(f"#{i}: Disk-shaped halo with {len(g):,} particles, mass = {halo_mass:.3e} Msun, position = {halo_center[0]:.3f}/{halo_center[1]:.3f}/{halo_center[2]:.3f}, %baryons = {baryon_count/len(g):.4f}")
        print(f"              CDM+baryons/baryons only")
        print(f"Radius:       {radius:.2f}/{radius_b:.2f} Mpc")
        print(f"Temp:         {av_baryon_temp:,.0f} K")
        print(f"|L|:          {np.linalg.norm(L_ang):.2e}/{np.linalg.norm(L_ang_b):.2e} Msun·Mpc²/Myr")
        print(f"|omega|:      {omega:.3e}/{omega_b:.3e} rad/Myr, {2*np.pi/1000/omega:.1f}/{2*np.pi/1000/omega_b:.1f} Gy per revolution")
        print(f"Triaxiality:  {triax:.3f}/{triax_b:.3f} (0=oblate, 1=prolate)")
        print(f"Overdensity:  {overdensity:.2f}/{overdensity_b:.2f}")
        print(f"Spin λ':      {spin:.4e}/{spin_b:.4e}")
        print()

__Visualize halos__

In [ ]:
# @title
# Additional Halo Analysis - Helpers
def minimum_image(dx, Lbox=None):
    # Apply minimum-image convention along last axis if Lbox is given.
    if Lbox is None:
        return dx
    return (dx + 0.5*Lbox) % Lbox - 0.5*Lbox

def shrink_center(x, m=None, Lbox=None, shrink=0.8, tol=1e-4, max_iter=50):
    # Iterative shrinking-sphere center finder (comoving).
    # Returns center (3,), indices kept.

    m = m * np.ones(x.shape[0])

    # start from mass-weighted COM
    c = (m[:,None] * x).sum(axis=0) / m.sum()
    # initial radius: half the max span
    dx = minimum_image(x - c, Lbox)
    r = np.linalg.norm(dx, axis=1)
    R = np.percentile(r, 90.0) if np.any(r>0) else 0.0

    keep = np.arange(x.shape[0])

    for _ in range(max_iter):
        if R <= 0:
            break
        # keep particles within R
        sel = r <= R
        if sel.sum() < 50:   # avoid pathological small-N
            break
        keep = keep[sel]
        # recompute center in that subset
        c_new = (m[keep,None]*x[keep]).sum(axis=0) / m[keep].sum()
        if np.linalg.norm(minimum_image(c_new - c, Lbox)) < tol * R:
            c = c_new
            break
        c = c_new
        # shrink sphere
        dx = minimum_image(x[keep] - c, Lbox)
        r = np.linalg.norm(dx, axis=1)
        R *= shrink

    return c, keep

def enclosed_200m(x, m, a, Omega_m, rho_crit0, center, Lbox=None):
    # Find R_200m and M_200m. Uses *comoving* positions and the 200×mean-matter threshold.
    # In comoving units, the threshold is constant: rho_bar,com = Omega_m * rho_crit0.
    # Condition: M(<R_com) / (4/3 π R_com^3) = 200 * Omega_m * rho_crit0

    m = np.asarray(m, dtype=np.float64)
    if m.ndim == 0:
        m = np.full(x.shape[0], float(m))

    rho_bar_com = Omega_m * rho_crit0  # [Msun/Mpc^3], constant in comoving coordinates

    # radii from center (comoving)
    dx = minimum_image(x - center, Lbox)
    r = np.linalg.norm(dx, axis=1)  # comoving Mpc
    order = np.argsort(r)
    r_sorted = r[order]
    m_sorted = m[order]
    M_enc = np.cumsum(m_sorted)

    with np.errstate(divide='ignore', invalid='ignore'):
        rho_enc_com = M_enc / ((4.0/3.0)*np.pi * np.maximum(r_sorted, 1e-12)**3)

    target = 200.0 * rho_bar_com
    # first radius where enclosed mean falls below target (profile declines outward)
    idx = np.searchsorted(rho_enc_com < target, True)
    if idx == r_sorted.size:
        # did not drop below target within the sample; take outermost
        idx_star = r_sorted.size - 1
    else:
        idx_star = idx

    R200m_com = r_sorted[idx_star]
    M200m     = M_enc[idx_star]
    R200m_phys = a * R200m_com
    return R200m_com, R200m_phys, M200m, order[:idx_star+1]  # indices inside R200m

def bullock_spin(x, u, m, a, G, center, indices_R, Lbox=None):
    # Bullock spin parameter λ' inside R200m.
    # Inputs: x (comoving), u (comoving peculiar), m, scale a, G in Mpc^3/(Msun Myr^2).

    m = np.asarray(m, dtype=np.float64)
    if m.ndim == 0:
        m = np.full(x.shape[0], float(m))

    I = indices_R
    mR = m[I]
    # relative comoving coords/vels to halo bulk
    x_rel = minimum_image(x[I] - center, Lbox)
    u_cm  = (mR[:,None] * u[I]).sum(axis=0) / mR.sum()
    u_rel = u[I] - u_cm

    # J_phys = a^2 * sum m (x × u)   (since r = a x, v_pec = a u)
    J_com = np.sum(np.cross(x_rel, u_rel) * mR[:,None], axis=0)
    J_phys = (a*a) * J_com
    Jnorm = np.linalg.norm(J_phys)

    # R and M inside selection
    r_phys = a * np.linalg.norm(x_rel, axis=1)
    R = r_phys.max()
    M = mR.sum()

    Vc = np.sqrt(G * M / R)
    lambda_bullock = Jnorm / (np.sqrt(2.0) * M * Vc * R)
    return lambda_bullock, J_phys, R, M

def radial_velocity_profile(x, u, m, a, center, R200m_phys, Lbox=None, edges=None, nbins=10, pct=(16,50,84)):
    # Radial velocity stats vs radius (physical). Uses peculiar velocities (no Hubble term).
    # Returns: r_bin_centers [Mpc], v_r percentiles [km/s-equivalent if you convert units].
    #Here units stay in whatever your u uses (Mpc/Myr).

    m = np.asarray(m, dtype=np.float64)
    if m.ndim == 0:
        m = np.full(x.shape[0], float(m))

    x_rel = minimum_image(x - center, Lbox)
    r_phys = a * np.linalg.norm(x_rel, axis=1)
    # physical peculiar velocities
    u_rel = u - (m[:,None]*u).sum(axis=0)/m.sum()  # subtract global CM (or recompute inside R if desired)
    v_phys = a * u_rel
    # radial component
    with np.errstate(invalid='ignore', divide='ignore'):
        rhat = np.divide(x_rel, np.linalg.norm(x_rel, axis=1)[:,None], where=(r_phys[:,None]>0))
    v_r = np.sum(v_phys * rhat, axis=1)

    # Binning by spherical radius
    if edges is None:
        edges = np.linspace(0.0, R200m_phys, nbins+1)
    centers = 0.5*(edges[:-1] + edges[1:])

    prc = np.empty((len(pct), nbins))
    for i in range(nbins):
        mask = (r_phys >= edges[i]) & (r_phys < edges[i+1])
        if np.any(mask):
            prc[:, i] = np.percentile(v_r[mask], pct)
        else:
            prc[:, i] = np.nan
    return centers, prc, edges

def rotation_profiles(x, u, t, is_baryon, m, a, center, R200m_phys, G, Lbox=None, edges=None, nbins=10, min_count=20):
    # x,u: comoving positions and peculiar velocities
    # m  : scalar or array [Msun]
    # a  : scale factor
    # center: comoving center
    # Returns dict with:
    #   - r_centers [Mpc]
    #   - mean_omega [rad/Myr], mean_vphi [km/s] (mass-weighted, signed)
    #   - med_vphi_pro [km/s] (median of prograde-only)
    #   - Vc [km/s] (circular velocity from enclosed mass)

    if np.ndim(m)==0:
        m = np.full(x.shape[0], float(m))
    m = m.astype(float)

    def mic(dx, L): return (dx + 0.5*L) % L - 0.5*L if L is not None else dx

    # Relative (comoving) and physical vectors
    dx = mic(x - center, Lbox)
    r_phys = a * np.linalg.norm(dx, axis=1)

    # Bulk peculiar velocity inside R200m
    inside = r_phys <= R200m_phys
    u_cm = (m[inside,None]*u[inside]).sum(axis=0)/m[inside].sum()

    r_vec = a * dx
    v_vec = a * (u - u_cm)

    # Spin axis from particles inside R200m
    J = np.sum(np.cross(r_vec[inside], v_vec[inside]) * m[inside,None], axis=0)
    n_hat = J/np.linalg.norm(J) if np.linalg.norm(J)>0 else np.array([0,0,1.0])

    # Cylindrical radius about spin axis and specific ang. mom. component
    eps = 1e-12
    R_perp = np.linalg.norm(np.cross(r_vec, n_hat), axis=1) # [Mpc]
    jz     = np.einsum('ij,j->i', np.cross(r_vec, v_vec), n_hat) # [Mpc^2/Myr]
    # component of v along azimuthal direction = jz / R_perp (but be careful at axis)
    vphi_signed = jz / np.maximum(R_perp, eps) # [Mpc/Myr]

    # Binning by spherical radius
    if edges is None:
        edges = np.linspace(0.0, R200m_phys, nbins+1)
    centers = 0.5*(edges[:-1] + edges[1:])

    mean_omega = np.full(len(centers), np.nan)      # [rad/Myr]
    mean_vphi  = np.full(len(centers), np.nan)      # [Mpc/Myr]
    med_vphi_pro = np.full(len(centers), np.nan)    # [Mpc/Myr]
    Vc = np.full(len(centers), np.nan)              # [Mpc/Myr]
    prograde_ratio = np.full(len(centers), np.nan)  # 1
    krot_b = np.full(len(centers), np.nan)  # 1
    mean_temp = np.full(len(centers), np.nan)  # K
    n_b = np.full(len(centers), np.nan)  # 1

    # Precompute cumulative mass for Vc
    order = np.argsort(r_phys)
    r_sorted = r_phys[order]
    M_cum = np.cumsum(m[order])
    # Helper to get enclosed mass at a radius via searchsorted
    def enclosed_mass(r):
        idx = np.searchsorted(r_sorted, r, side='right')-1
        if idx < 0: return 0.0
        return M_cum[idx]

    for i in range(len(centers)):
        mask = (r_phys >= edges[i]) & (r_phys < edges[i+1]) & is_baryon
        n_b[i] = mask.sum()
        if n_b[i] < min_count: continue

        mean_temp[i] = np.sum(m[mask]*t[mask] / np.sum(m[mask]))

        # Mass-weighted mean omega: sum(jz)/sum(m R_perp^2)
        denom = (m[mask] * np.maximum(R_perp[mask]**2, eps**2)).sum()
        num   = jz[mask].sum()
        if denom > 0:
            mean_omega[i] = num / denom  # [rad/Myr]
            mean_vphi[i]  = (mean_omega[i] * (m[mask]*R_perp[mask]).sum()/m[mask].sum()) # [Mpc/Myr]

        # kappa_rot for baryons only in this bin
        v2_b = np.sum(v_vec[mask]**2, axis=1) # [Mpc^2/Myr^2]
        Ktot_b = 0.5 * np.sum(m[mask] * v2_b)
        Krot_b = 0.5 * np.sum(m[mask] * (vphi_signed[mask]**2))
        krot_b[i] = Krot_b / Ktot_b if Ktot_b > 0 else np.nan

        # Prograde-only median azimuthal speed
        pro = mask & (jz > 0)
        if pro.sum() >= min_count:
            med_vphi_pro[i] = np.median(vphi_signed[pro])
            prograde_ratio[i] = pro.sum() / mask.sum()

        # Circular velocity from enclosed mass at bin center
        Menc = enclosed_mass(centers[i])
        if Menc > 0 and centers[i] > 0:
            Vc[i] = np.sqrt(G * Menc / centers[i]) # [Mpc/Myr]

    return centers, mean_omega, mean_vphi, med_vphi_pro, prograde_ratio, krot_b, Vc, n_hat, mean_temp, n_b

In [ ]:
# @title
# Additional Halo Analysis - Main function
def analyze_halo_200m(x, u, t, is_baryon, m, a, Omega_m, rho_crit0, G, Lbox=None, nbins=25):
    # Full analysis:
    #  - center (shrink sphere)
    #  - R200m, M200m
    #  - Bullock spin inside R200m
    #  - radial velocity profile (to 2*R200m)

    center, _ = shrink_center(x, m, Lbox=Lbox) # 1) center
    R200m_com, R200m_phys, M200m, idx_in = enclosed_200m(x, m, a, Omega_m, rho_crit0, center, Lbox=Lbox) # 2) R200m, M200m
    lam_p, Jvec, Rsel, Msel = bullock_spin(x, u, m, a, G, center, idx_in, Lbox=Lbox) # 3) spin inside R200m
    r_centers, v_prc, r_edges = radial_velocity_profile(x, u, m, a, center, 0.3*R200m_phys, edges=None, Lbox=Lbox, nbins=nbins) # 4) radial velocity profile out to 2*R200m
    r_centers, mean_omega, mean_vphi, med_vphi_pro, prograde_ratio, k_rot, Vc, n_hat, mean_temp, n_b = rotation_profiles(x, u, t, is_baryon, m, a, center, 0.3*R200m_phys, G, Lbox=None, nbins=nbins, min_count=10, edges=None)
    mask = ~np.isnan(med_vphi_pro) & ~np.isnan(Vc)
    Vc_sum = np.sum(Vc[mask])
    rot_support_ratio = np.sum(med_vphi_pro[mask]) / Vc_sum if Vc_sum != 0.0 else np.nan

    out = {
        "center_com": center,
        "R200m_com": R200m_com,
        "R200m_phys": R200m_phys,
        "M200m": M200m,
        "spin_lambda_prime": lam_p,
        "J_phys": Jvec,
        "rv_radii_phys": r_centers,
        "rv_percentiles": {"p16": v_prc[0], "p50": v_prc[1], "p84": v_prc[2]},
        "rv_edges_phys": r_edges,
        "indices_inside_R200m": idx_in,
        "mean_omega": mean_omega,
        "mean_vphi": mean_vphi,
        "med_vphi_pro": med_vphi_pro,
        "prograde_ratio": prograde_ratio,
        "k_rot": k_rot,
        "Vc": Vc,
        "n_hat": n_hat,
        "mean_temp": mean_temp,
        "rot_support_ratio": rot_support_ratio,
        "n_b": n_b
    }
    return out

def analyze_disk_region(
    x, u, t, is_baryon, m, a, G,
    f_spin=0.1,                # inner sphere (in units of R200m) to define spin axis
    f_R=0.2,                   # cylinder radius = f_R * R200m (physical)
    f_z=0.05,                  # cylinder half-thickness = f_z * R200m (physical)
    nbins=8,                   # radial bins within the cylinder
    min_count=20               # minimum particles per selection/bin
):
    """
    x,u: (N,3) comoving positions [Mpc], peculiar velocities [Mpc/Myr]
    m  : scalar or (N,) masses [Msun]
    is_baryon: (N,) boolean mask for baryons
    a  : scale factor
    center: (3,) comoving halo center [Mpc]
    R200m_phys: halo radius [Mpc physical]

    Returns dict with:
      n_hat, disk_masks, kappa_rot_baryon_disk, f_prograde_baryon_disk,
      rbin_centers_phys, med_vphi_pro_baryon_kms, Vc_total_kms,
      kappa_rot_baryon_bins, f_prograde_baryon_bins
    """
    N = x.shape[0]
    if np.ndim(m) == 0:
        m = np.full(N, float(m))
    m = m.astype(float)

    center, _ = shrink_center(x, m, Lbox=None) # center
    _, R200m_phys, M200m, _ = enclosed_200m(x, m, a, Ω_m, rho_crit0, center, Lbox=None) # R200m, M200m

    # --- relative physical positions & velocities ---
    dx = x - center      # comoving
    r_phys = a * dx                 # [Mpc]
    v_phys = a * u                  # [Mpc/Myr]

    # --- robust spin axis from baryons in inner sphere ---
    R_spin = max(1e-6, f_spin * R200m_phys)
    r_sph = np.linalg.norm(r_phys, axis=1)
    sel_spin = is_baryon & (r_sph <= R_spin)
    if sel_spin.sum() < 5:
        # fallback: use all baryons
        sel_spin = is_baryon

    L_vec = np.sum(m[sel_spin, None] * np.cross(r_phys[sel_spin], v_phys[sel_spin]), axis=0)
    L_norm = np.linalg.norm(L_vec)
    n_hat = L_vec / L_norm if L_norm > 0 else np.array([0.0, 0.0, 1.0])

    # --- cylindrical coordinates around n_hat ---
    z = np.einsum('ij,j->i', r_phys, n_hat)                         # [Mpc]
    R_perp = np.linalg.norm(np.cross(r_phys, n_hat), axis=1)        # [Mpc]
    jz = np.einsum('ij,j->i', np.cross(r_phys, v_phys), n_hat)      # [Mpc^2/Myr]
    vphi = jz / np.maximum(R_perp, 1e-12)                           # [Mpc/Myr]

    # --- disk selection (baryons only) ---
    R_max = f_R * R200m_phys
    z_max = f_z * R200m_phys
    disk_mask = (np.abs(z) <= z_max) & (R_perp <= R_max)
    disk_baryons = disk_mask & is_baryon

    # --- global disk diagnostics (baryons only) ---
    out = {"n_hat": n_hat, "disk_mask": disk_mask, "disk_baryons": disk_baryons}

    if disk_baryons.sum() >= min_count:
        v2_b = np.sum(v_phys[disk_baryons]**2, axis=1)                 # [Mpc^2/Myr^2]
        Ktot_b = 0.5 * np.sum(m[disk_baryons] * v2_b)
        Krot_b = 0.5 * np.sum(m[disk_baryons] * (vphi[disk_baryons]**2))
        kappa_rot_baryon_disk = Krot_b / Ktot_b if Ktot_b > 0 else np.nan
        f_prograde_baryon_disk = np.count_nonzero(jz[disk_baryons] > 0) / disk_baryons.sum()
    else:
        kappa_rot_baryon_disk = np.nan
        f_prograde_baryon_disk = np.nan

    out["kappa_rot_baryon_disk"] = kappa_rot_baryon_disk
    out["f_prograde_baryon_disk"] = f_prograde_baryon_disk

    # --- Vc(r): spherical enclosed mass using ALL mass (DM + baryons) ---
    r_all = np.linalg.norm(r_phys, axis=1)
    order = np.argsort(r_all)
    r_sorted = r_all[order]
    Mcum = np.cumsum(m[order])

    def M_enclosed(r0):
        i = np.searchsorted(r_sorted, r0, side='right') - 1
        return 0.0 if i < 0 else Mcum[i]

    # --- radial bins in the disk (cylindrical radius) ---
    if nbins is None or nbins < 1:
        return out

    # Only consider disk area up to R_max
    edges = np.linspace(0.0, R_max, nbins + 1)
    centers = 0.5 * (edges[:-1] + edges[1:])

    med_vphi_pro = np.full(nbins, np.nan)
    Vc_total = np.full(nbins, np.nan)
    krot_bins = np.full(nbins, np.nan)
    fpro_bins = np.full(nbins, np.nan)
    n_bins = np.full(nbins, np.nan)

    # Per-bin stats (baryons in disk)
    for i in range(nbins):
        sel_bin = disk_baryons & (R_perp >= edges[i]) & (R_perp < edges[i+1])
        n_bins[i] = sel_bin.sum()

        if sel_bin.sum() < min_count: continue

        # prograde fraction & median vphi (prograde baryons)
        pro = sel_bin & (jz > 0)
        fpro_bins[i] = np.count_nonzero(pro) / sel_bin.sum()
        if pro.any():
            med_vphi_pro[i] = np.median(vphi[pro])

        # kappa_rot in this ring (baryons)
        v2_b = np.sum(v_phys[sel_bin]**2, axis=1)
        Ktot_b = 0.5 * np.sum(m[sel_bin] * v2_b)
        Krot_b = 0.5 * np.sum(m[sel_bin] * (vphi[sel_bin]**2))
        krot_bins[i] = Krot_b / Ktot_b if Ktot_b > 0 else np.nan

        # benchmark circular speed from enclosed TOTAL mass at spherical r = bin center
        r0 = centers[i]
        if r0 > 0:
            Menc = M_enclosed(r0)
            if Menc > 0:
                Vc_total[i] = np.sqrt(G * Menc / r0)

    out.update({
        "rbin_centers_phys": centers,
        "med_vphi_pro_baryon_kms": med_vphi_pro,
        "Vc_total_kms": Vc_total,
        "kappa_rot_baryon_bins": krot_bins,
        "f_prograde_baryon_bins": fpro_bins,
    })

    print(f"R200m phys:         {R200m_phys:.3f} Mpc (CDM+baryons)")
    print(f"M200m:              {M200m:.3e} Msun (CDM+baryons)")
    #print(f"Spin Lambda prime:  {disk_char['spin_lambda_prime']:.3e} (CDM+baryons)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in centers) + '\t]'
    print(f"Radial bin centers: {formatted_list} Mpc")
    formatted_list = '[ ' + ',\t'.join(f"{r:,.0f}" for r in n_bins) + '\t]'
    print(f"number of baryons:  {formatted_list}")
    #formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['mean_temp']/1000) + '\t]'
    #print(f"Mean temperature:   {formatted_list} kK")
    #formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['rv_percentiles']['p50'] * Mpc_km/Myr_s) + '\t]'
    #print(f"Med radial vel:     {formatted_list} km/s (CDM+baryons)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in fpro_bins) + '\t]'
    print(f"Prograde fraction:  {formatted_list} (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in krot_bins) + '\t]'
    print(f"k_rot ratio:        {formatted_list} (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in med_vphi_pro * Mpc_km/Myr_s) + '\t]'
    print(f"Med vphi prograde:  {formatted_list} km/s (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in Vc_total * Mpc_km/Myr_s) + '\t]'
    print(f"Vc:                 {formatted_list} km/s (benchmark for med vphi prograde)")
    #print(f"Rot support ratio:  {halo_char['rot_support_ratio']:.3f} (baryons only)")

    return out


# if not np.isnan(r)
def print_halo_characteristics(halo_char):
    print(f"R200m phys:         {halo_char['R200m_phys']:.3f} Mpc (CDM+baryons)")
    print(f"M200m:              {halo_char['M200m']:.3e} Msun (CDM+baryons)")
    print(f"Spin Lambda prime:  {halo_char['spin_lambda_prime']:.3e} (CDM+baryons)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in halo_char['rv_radii_phys']) + '\t]'
    print(f"Radial bin centers: {formatted_list} Mpc")
    formatted_list = '[ ' + ',\t'.join(f"{r:,.0f}" for r in halo_char['n_b']) + '\t]'
    print(f"number of baryons:  {formatted_list}")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['mean_temp']/1000) + '\t]'
    print(f"Mean temperature:   {formatted_list} kK")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['rv_percentiles']['p50'] * Mpc_km/Myr_s) + '\t]'
    print(f"Med radial vel:     {formatted_list} km/s (CDM+baryons)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in halo_char['prograde_ratio']) + '\t]'
    print(f"Prograde fraction:  {formatted_list} (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.3f}" for r in halo_char['k_rot']) + '\t]'
    print(f"k_rot ratio:        {formatted_list} (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['med_vphi_pro'] * Mpc_km/Myr_s) + '\t]'
    print(f"Med vphi prograde:  {formatted_list} km/s (baryons only)")
    formatted_list = '[ ' + ',\t'.join(f"{r:.1f}" for r in halo_char['Vc'] * Mpc_km/Myr_s) + '\t]'
    print(f"Vc:                 {formatted_list} km/s (benchmark for med vphi prograde)")
    print(f"Rot support ratio:  {halo_char['rot_support_ratio']:.3f} (baryons only)")



In [ ]:
# @title
# Visualize halo
halo_idx = 0
if len(groups) < halo_idx+1:
    print(f"There are less than {halo_idx} halos")

halo_N_bodies = len(groups[halo_idx])
halo_pos = final_pos[groups[halo_idx]]
halo_vel = final_vel[groups[halo_idx]]
halo_masses = masses[groups[halo_idx]]
halo_temp = final_temp[groups[halo_idx]]
halo_baryons = np.intersect1d(groups[halo_idx], baryon_indices)

halo_radius, L_ang, omega, overdensity, spin, triax = analyze_halo(halo_pos, halo_vel, halo_masses, scale_hist[-1], G)
x_cm = np.mean(halo_pos, axis=0)
v_cm = np.mean(halo_vel, axis=0)
x_rel = (halo_pos - x_cm) * scale
halo_baryon_indices = np.intersect1d(baryon_indices, groups[halo_idx])
halo_CDM_indices = np.setdiff1d(groups[halo_idx], baryon_indices)
x_rel_baryon = (final_pos[halo_baryon_indices] - x_cm) * scale
x_rel_CDM = (final_pos[halo_CDM_indices] - x_cm) * scale

print(f"Halo #{halo_idx}: total mass = {halo_masses.sum():.2e} m_sun, position = {x_cm[0]:.3f}/{x_cm[1]:.3f}/{x_cm[2]:.3f}")

halo_char = analyze_halo_200m(halo_pos, halo_vel, halo_temp, baryon_set[groups[halo_idx]], halo_masses, scale, Ω_m, rho_crit0, G, Lbox=None, nbins=5)
print_halo_characteristics(halo_char)
#print()
#analyze_disk_region(halo_pos, halo_vel, halo_temp, baryon_set[groups[halo_idx]], masses[groups[halo_idx]], scale, G, nbins=5)

R_scale = halo_radius
fig = go.Figure(
    data=[
        go.Scatter3d(x=x_rel_CDM[:, 0], y=x_rel_CDM[:, 1], z=x_rel_CDM[:, 2], mode='markers', marker=dict(color='blue', size=1), name='CDM'),
        go.Scatter3d(x=x_rel_baryon[:, 0], y=x_rel_baryon[:, 1], z=x_rel_baryon[:, 2], mode='markers', marker=dict(color='red', size=1), name='Baryon'),
    ],
    layout=go.Layout(
        title=f'Halo plot #{halo_idx} ({halo_N_bodies:,} bodies)',
        width = 800,
        height = 600,
        scene=dict(
            xaxis=dict(title='x [Mpc]', range=[-halo_radius, halo_radius]),
            yaxis=dict(title='y [Mpc]', range=[-halo_radius, halo_radius]),
            zaxis=dict(title='z [Mpc]', range=[-halo_radius, halo_radius]),
            camera=dict(
                eye=dict(x=1.5, y=-1.5, z=1.5),
                up=dict(x=0, y=0, z=-1),
                projection=dict(type='perspective')
            ),
            aspectratio=dict(x=1.2, y=1.2, z=1.2)
        ),
        updatemenus=menues,
    )
)
fig.show()
#fig.write_html('Halo plot.html') # Save to HTML
halo_plot_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

In [ ]:
# @title
R_scale = L/2
phys_scale = True
body_pos = positions_hist[-1].astype(np.float16)
baryon_pos = body_pos[baryon_set[grid_downsampled]]
body_temp = temp_hist[-1]
baryon_temp = body_temp[baryon_set[grid_downsampled]]
cmin = 0
cmax = np.percentile(baryon_temp, 95)
ref_sphere = create_sphere_mesh(x_cm - center, 2*halo_radius, color='black')

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=baryon_pos[:, 0] * aa,
            y=baryon_pos[:, 1] * aa,
            z=baryon_pos[:, 2] * aa,
            mode='markers',
            marker=dict(
                size=2,
                color=baryon_temp,
                colorscale='plasma', # 'Viridis'
                cmin=cmin,
                cmax=cmax,
                colorbar=dict(title="Temp [K]"),
                opacity=0.8
            ),
            name='Baryon'
        ),
        ref_sphere
    ],
    layout=go.Layout(
        title=f'Halo #{halo_idx} identified in box',
        width=800,
        height=600,
        scene=dict(
            xaxis=dict(title='x [Mpc]', range=[-R_scale, R_scale]),
            yaxis=dict(title='y [Mpc]', range=[-R_scale, R_scale]),
            zaxis=dict(title='z [Mpc]', range=[-R_scale, R_scale]),
            camera=dict(
                eye=dict(x=0, y=0, z=2),
                up=dict(x=0, y=1, z=-0),
                projection=dict(type='orthographic')
            ),
            aspectratio=dict(x=1.8, y=1.8, z=1.8)
        ),
        updatemenus=menues,
    ),
)
fig.show()

In [ ]:
# @title
halo_set = np.zeros(N_bodies, dtype=bool)
halo_set[groups[halo_idx]] = True

frames = []
for i, pos in enumerate(positions_hist):
    aa = scale_hist[-1]
    pos = pos.astype(np.float16)
    ref_sphere = create_sphere_mesh(np.zeros(3), R_scale * scale_hist[i], color='gray')
    frames.append(go.Frame(
        data=[
            go.Scatter3d(x=pos[~halo_set[grid_downsampled], 0]*aa, y=pos[~halo_set[grid_downsampled], 1]*aa, z=pos[~halo_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
            go.Scatter3d(x=pos[halo_set[grid_downsampled], 0]*aa, y=pos[halo_set[grid_downsampled], 1]*aa, z=pos[halo_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=2), name='Baryon'),
        ],
        name=f'frame{i}',
        layout=go.Layout(title_text='Evolution of halo')
    ))

# Initial frame
init_pos = positions_hist[0].astype(np.float16)
aa = scale_hist[-1]
fig = go.Figure(
    data=[
        go.Scatter3d(x=init_pos[~halo_set[grid_downsampled], 0]*aa, y=init_pos[~halo_set[grid_downsampled], 1]*aa, z=init_pos[~halo_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
        go.Scatter3d(x=init_pos[halo_set[grid_downsampled], 0]*aa, y=init_pos[halo_set[grid_downsampled], 1]*aa, z=init_pos[halo_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=2), name='Baryon'),
    ],
    layout=go.Layout(
        title='Evolution of halo',
        width = 1000,
        height = 800,
        scene=dict(
            xaxis=dict(title='x [Mpc]', range=[-R_scale, R_scale]),
            yaxis=dict(title='y [Mpc]', range=[-R_scale, R_scale]),
            zaxis=dict(title='z [Mpc]', range=[-R_scale, R_scale]),
            camera=dict(
                eye=dict(x=1.4, y=-1.8, z=1.3),
                up=dict(x=0, y=0, z=-1),
                projection=dict(type='perspective')
            ),
            aspectratio=dict(x=1.2, y=1.2, z=1.2)
        ),
        updatemenus=menues,
        sliders=[{
                    "steps": [
                        {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                         "label": f"{time_hist[i]:,.0f} Myr",
                         "method": "animate"
                         } for i in range(0,len(positions_hist),1)
                    ],
                }],
    ),
    frames=frames
)
fig.show()
#fig.write_html('DFS-v9-comoving.html') # Save to HTML
#plot_html = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')


__Find halos with high rotational support__

In [ ]:
# @title
for i, g in enumerate(groups):
    if i>100: break
    halo_pos = final_pos[g]
    halo_vel = final_vel[g]
    halo_masses = masses[g]
    halo_temp = final_temp[g]
    halo_char = analyze_halo_200m(halo_pos, halo_vel, halo_temp, baryon_set[g], halo_masses, scale, Ω_m, rho_crit0, G, Lbox=None, nbins=10)
    radius, L_ang, omega, overdensity, spin, triax = analyze_halo(halo_pos, halo_vel, halo_masses, scale_hist[-1], G) # CDM+baryons
    if halo_char['rot_support_ratio'] > 0.9:
        print(f"#{i}: Disk-shaped halo with {len(g):,} and triaxiality {triax:.3f}")
        print_halo_characteristics(halo_char)
        print()

__Layzer-Ervine Energy Equation__

In [ ]:
# @title
def layzer_irvine(t, a, K, U):
    H = np.gradient(a, t) / a
    dE_dt = np.gradient(K+U, t)
    resid = dE_dt + H*(2*K+U)           # should be ~ 0
    return resid, dE_dt, H*(2*K+U)

t_array = np.array(time_hist)
a_array = np.array(scale_hist)
K_array = np.array(KE_hist)
U_array = np.array(PE_hist)
resid, dE_dt, H2KpU = layzer_irvine(t_array, a_array, K_array, U_array)

fig, ax = plt.subplots(figsize=(7, 4))  # no longer (2,1)
ax.set_title("Layzer-Irvine")
ax.plot(t_array, resid, label=r"$\frac{d}{dt}(K+U)+H(2K+U)$")
ax.plot(t_array, dE_dt, label=r"$\frac{d}{dt}(K+U)$")
ax.plot(t_array, H2KpU, label=r"$H(2K+U)$")
ax.axhline(0, color='k', lw=0.7)
ax.legend()
ax.grid(ls=':')
ax.set_xlabel("Time [Myr]")
ax.set_ylabel("LI residual [M$_\\odot$⋅Mpc²/Myr³]")

plt.tight_layout()
plt.show()
LI_html = create_html(fig)

__Weak-lensing convergence kappa__

In [ ]:
# @title
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d

# -----------------------------
# Cosmology helpers
# -----------------------------
def E_z(z):
    return E(1/(1+z))

def comoving_chi_of_z(z, H0=67.5, **cosmo):
    """
    chi(z) in Mpc (comoving), using c/H integral.
    """
    c = 299792.458  # km/s
    z_grid = np.linspace(0.0, z, max(2000, int(400*z+200)))  # dense enough
    Ez = E_z(z_grid)
    chi_grid = cumulative_trapezoid(c / (H0 * Ez), z_grid, initial=0.0)  # Mpc
    return z_grid, chi_grid

def make_z_of_chi(z_max=10.0, H0=67.5, **cosmo):
    z_grid = np.linspace(0.0, z_max, 5000)
    c = 299792.458
    chi_grid = cumulative_trapezoid(c / (H0 * E_z(z_grid)), z_grid, initial=0.0)
    chi_max = chi_grid[-1]
    z_of_chi = interp1d(chi_grid, z_grid, kind='linear', bounds_error=False,
                        fill_value=(z_grid[0], z_grid[-1]))
    return z_of_chi, chi_max

# -----------------------------
# Kappa from a 3D delta slab
# -----------------------------
def kappa_from_delta_slab(delta_zyx, Lbox, z_start, z_source,
                          H0=67.5, Om=0.315, Ol=0.685, Or=0.0):
    """
    Parameters
    ----------
    delta_zyx : ndarray, shape (Nz, Ny, Nx)
        3D overdensity cube δ(χ, y, x) at (approximately) the same epoch
        across the slab. Periodic boundary not required.
    Lbox : float
        Comoving box size along each axis, in Mpc units.
    z_start : float
        Redshift at the *front* face of the slab (closest to observer).
    z_source : float
        Source redshift for the lensing kernel (all sources at one plane).
    H0, Om, Ol, Or : cosmology

    Returns
    -------
    kappa : ndarray, shape (Ny, Nx)
        Convergence map (dimensionless).
    info  : dict
        Useful metadata (chi_s, slice chis, weights, etc.)
    """
    Nz, Ny, Nx = delta_zyx.shape
    dchi = Lbox / Nz  # Mpc per slice

    # Distances for source and slab
    z_of_chi, chi_max = make_z_of_chi(z_max=max(z_source, z_start) + 2.0, H0=H0, Om=Om, Ol=Ol, Or=Or)
    # chi at front face:
    z_grid_front, chi_front_arr = comoving_chi_of_z(z_start, H0=H0, Om=Om, Ol=Ol, Or=Or)
    chi_front = chi_front_arr[-1]
    # chi of source:
    _, chi_s_arr = comoving_chi_of_z(z_source, H0=H0, Om=Om, Ol=Ol, Or=Or)
    chi_s = chi_s_arr[-1]

    # LOS comoving coordinate of slice centers
    chi_slices = chi_front + (np.arange(Nz) + 0.5) * dchi
    # Convert each slice chi -> z_i and a_i
    z_slices = z_of_chi(chi_slices)
    a_slices = 1.0 / (1.0 + z_slices)

    # Lensing kernel per slice (Born approx)
    geom = chi_slices * (chi_s - chi_slices) / chi_s
    geom = np.clip(geom, 0.0, None)  # zero outside [0, chi_s]

    # Prefactor
    c = 299792.458  # km/s
    pref = 1.5 * Om * (H0 / c)**2  # dimensionless per Mpc factor in integral

    # Perform weighted LOS sum: κ(y,x) = pref * Σ_i [ δ_i(y,x) * dchi * geom_i / a_i ]
    weights = (geom / a_slices) * dchi  # Mpc
    # reshape for broadcasting: (Nz,1,1)
    w = weights[:, None, None]
    kappa = pref * np.sum(delta_zyx * w, axis=0)

    info = dict(chi_s=float(chi_s), z_slices=z_slices, chi_slices=chi_slices,
                a_slices=a_slices, weights=weights, pref=pref, dchi=dchi)
    return kappa, info

# Place slab starting at z=0.3, sources at z_s=1.0
kappa, info = kappa_from_delta_slab(delta_end.transpose(2,0,1),  # -> (Z,X,Y)
                                    Lbox=L,
                                    z_start=0.3, z_source=1.0,
                                    H0=H0, Om=Ω_m, Ol=Ω_Λ, Or=Ω_r)

# Plot κ-map (single chart, default colors)
vmax = np.percentile(kappa, 98)
fig = plt.figure(figsize=(6,5))
plt.imshow(kappa.T, origin="lower", extent=[-L/2, L/2, -L/2, L/2], vmax=vmax)
plt.colorbar(label="κ")
plt.xlabel("x [Mpc]")
plt.ylabel("y [Mpc]")
plt.title("Weak-lensing convergence κ")
plt.tight_layout()
plt.show()
weaklensing_html = create_html(fig)

__Halo development over time__

In [ ]:
# @title
# define refined grid aligned to coarse cells (choose integer cell indices)
# choose coarse indices that define cube: find range of coarse cells covering cube
if cube_fine is not None:
    dx=L/Ng
    i_min = int(((center_fine[0]-cube_fine/2.0) / L) * Ng+1)
    j_min = int(((center_fine[1]-cube_fine/2.0) / L) * Ng+1)
    k_min = int(((center_fine[2]-cube_fine/2.0) / L) * Ng+1)
    n_cells = int((cube_fine / L) * Ng)  # number of coarse cells per side

    # Refine grid
    Nf = n_cells * refinement
    box_min_fine = np.array([i_min*dx, j_min*dx, k_min*dx]) # fine box min/max in physical coords
    box_max_fine = box_min_fine + n_cells*dx
    dx_fine = dx / refinement

    # Mask particles within fine box and deposit to fine rho grid
    mask_in_cube = np.all((final_pos >= box_min_fine) & (final_pos < box_max_fine), axis=1)
    pos_in = final_pos[mask_in_cube]
    pos_in.shape

    # Create Plotly animation frames
    frames = []
    for i, pos in enumerate(positions_hist):
        aa = scale_hist[-1]
        pos = pos.astype(np.float16)
        frames.append(go.Frame(
            data=[
                go.Scatter3d(x=pos[~baryon_set[grid_downsampled], 0]*aa, y=pos[~baryon_set[grid_downsampled], 1]*aa, z=pos[~baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
                go.Scatter3d(x=pos[baryon_set[grid_downsampled], 0]*aa, y=pos[baryon_set[grid_downsampled], 1]*aa, z=pos[baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=2), name='Baryon'),
            ],
            name=f'frame{i}',
            layout=go.Layout(title_text=f'Halo #{halo_idx} development over time (reduced resolution)')
        ))

    # Initial frame
    init_pos = positions_hist[0].astype(np.float16)
    aa = scale_hist[-1]
    fig = go.Figure(
        data=[
            go.Scatter3d(x=init_pos[~baryon_set[grid_downsampled], 0]*aa, y=init_pos[~baryon_set[grid_downsampled], 1]*aa, z=init_pos[~baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
            go.Scatter3d(x=init_pos[baryon_set[grid_downsampled], 0]*aa, y=init_pos[baryon_set[grid_downsampled], 1]*aa, z=init_pos[baryon_set[grid_downsampled], 2]*aa, mode='markers', marker=dict(color='red', size=2), name='Baryon'),
        ],
        layout=go.Layout(
            title=f'Halo #{halo_idx} development over time (reduced resolution)',
            width = 800,
            height = 600,
            scene=dict(
                xaxis=dict(title='x [Mpc]', range=[box_min_fine[0]-L/2, box_max_fine[0]-L/2]),
                yaxis=dict(title='y [Mpc]', range=[box_min_fine[1]-L/2, box_max_fine[1]-L/2]),
                zaxis=dict(title='z [Mpc]', range=[box_min_fine[2]-L/2, box_max_fine[2]-L/2]),
                camera=dict(
                    eye=dict(x=1.4, y=-1.8, z=1.3),
                    up=dict(x=0, y=0, z=-1),
                    projection=dict(type='perspective')
                ),
                aspectratio=dict(x=1.2, y=1.2, z=1.2)
            ),
            updatemenus=menues,
            sliders=[{
                        "steps": [
                            {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                            "label": f"{time_hist[i]:,.0f} Myr",
                            "method": "animate"
                            } for i in range(0,len(positions_hist),1)
                        ],
                    }],
        ),
        frames=frames
    )
    fig.show()

In [ ]:
# @title
# Create Plotly animation frames

if "box_max_fine" in globals():
    init_pos = final_pos.astype(np.float16)
    halo_mask = (init_pos[:,0] >= box_min_fine[0]) & (init_pos[:,0] < box_max_fine[0]) & (init_pos[:,1] >= box_min_fine[1]) & (init_pos[:,1] < box_max_fine[1]) & (init_pos[:,2] >= box_min_fine[2]) & (init_pos[:,2] < box_max_fine[2])
    init_pos = init_pos[halo_mask]

    aa = scale_hist[-1]
    fig = go.Figure(
        data=[
            go.Scatter3d(x=init_pos[~baryon_set[halo_mask], 0]*aa, y=init_pos[~baryon_set[halo_mask], 1]*aa, z=init_pos[~baryon_set[halo_mask], 2]*aa, mode='markers', marker=dict(color='blue', size=1), name='CDM'),
            go.Scatter3d(x=init_pos[baryon_set[halo_mask], 0]*aa, y=init_pos[baryon_set[halo_mask], 1]*aa, z=init_pos[baryon_set[halo_mask], 2]*aa, mode='markers', marker=dict(color='red', size=2), name='Baryon'),
        ],
        layout=go.Layout(
            title=f'Neighborhood halo #{halo_idx} ({init_pos.shape[0]:,} bodies)',
            width = 800,
            height = 600,
            scene=dict(
                xaxis=dict(title='x [Mpc]', range=[box_min_fine[0], box_max_fine[0]]),
                yaxis=dict(title='y [Mpc]', range=[box_min_fine[1], box_max_fine[1]]),
                zaxis=dict(title='z [Mpc]', range=[box_min_fine[2], box_max_fine[2]]),
                camera=dict(
                    eye=dict(x=1.4, y=-1.8, z=1.3),
                    up=dict(x=0, y=0, z=-1),
                    projection=dict(type='perspective')
                ),
                aspectratio=dict(x=1.2, y=1.2, z=1.2)
            ),
            updatemenus=menues,
        ),
    )
    fig.show()

__Create summary report__

In [ ]:
# @title
# Create summary report
import html
import zipfile

try:
    fine_grid_od_html
except NameError:
    fine_grid_od_html = ""

escaped_report = html.escape(report_text)
report_html = f'<details open><summary><strong>📄 Simulation Report (click to collapse)</strong></summary><pre style="font-family: monospace; font-size: 12px; background: #f0f0f0; padding: 10px;">{escaped_report}</pre></details>'
full_report = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Simulation Report</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>

{plot_html}    <!-- the plotly animation -->
{report_html}  <!-- your custom text goes here -->
{thermo_animation_html}
{halo_plot_html}    <!-- the plotly animation -->
{deltafield_html}
{thermo_html}
{energy_html}
{LI_html}
{fine_grid_od_html}
{weaklensing_html}

</body>
</html>
"""

with zipfile.ZipFile("DFS-v11-1-summary.zip", "w", compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.writestr("DFS-v11-1-summary.html", full_report)

"""
with open("DFS-v11-1-summary.html", "w") as f:
    f.write(minified_report)
""";

__Save simulation result to database__

In [ ]:
# @title
# Save simulations reesult to database
parameters = {
    "L": L,
    "N": N,
    "qp": qp,
    "qpa": qpa,
    "q_grid": qg,
    "steps": steps,
    "steps_fine": steps_fine,
    "h": h,
    "Ω_r": Ω_r,
    "Ω_γ": Ω_γ,
    "Ω_m": Ω_m,
    "Ω_b": Ω_b,
    "Ω_Λ": Ω_Λ,
    "Ω_k": Ω_k,
    "a_init": a_init,
    "a_final": a_final,
    "seed": seed,
    "use_TSC": use_TSC,
}

np.savez_compressed("DFS-v11-1-results.npz",
    parameters=parameters,
    sim_log = sim_log.getvalue(),
    final_pos=final_pos,
    final_vel=final_vel,
    final_temp=final_temp,
    final_pressure = final_pressure,
    masses=masses,
    baryon_set=baryon_set,
    baryon_indices=baryon_indices,
    grid_downsampled = grid_downsampled,
    positions_hist = positions_hist,
    scale_hist = scale_hist,
    time_hist = time_hist,
    temp_hist = temp_hist,
    pressure_hist = pressure_hist,
    KE_hist = KE_hist,
    PE_hist = PE_hist,
    TE_hist = TE_hist,
    virial_hist = virial_hist,
    flat_a_hist = flat_a_hist,
    flat_b_hist = flat_b_hist,
    flat_c_hist = flat_c_hist
)

In [ ]:
# @title
# Call of combined_step_py(..) for debugging purposes
"""
H_a = H0_cos * E(scale) #1/Myr
dt = np.clip(dt_target, epsilon_fine / H_a, epsilon / H_a)  # Myr

(body_positions, body_velocities_cos, acc, scale, PE,
u, T, acc_P, P, diag, epochs, dt_target_new,
max_b_od, max_tot_od, temp_by_b_od, S_max) = combined_step_subcycle_py(
    x=body_positions,
    u=body_velocities_cos,
    g=acc,
    gP_b=acc_P,
    U_b=u,
    baryon_mask=baryon_set,
    mass=mass,
    L=L,
    Ng=Ng,
    dt=dt,
    a=scale,
    G=G,
    eng=eng,
    use_TSC=use_TSC,
    n_threads=n_threads
)
""";